In [1]:
import autograd.numpy as np
import autograd.scipy as scipy
# import numpy as np
# import scipy
import scipy.linalg
import scipy.cluster
import scipy.stats
from autograd import grad, elementwise_grad, jacobian, hessian, value_and_grad
from collections import OrderedDict
import random

import plotly
from plotly.offline import iplot as plt
from plotly import graph_objs as plt_type
plotly.offline.init_notebook_mode(connected=True)

from IPython.core.debugger import set_trace



# Kernel function

In [3]:
def square_dist(x, x2=None, lengthscales=None):
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    
    x = x / lengthscales
    xs = np.sum(np.square(x), axis=0)
    if x2 is None:
        return -2 * np.dot(x.T, x) + \
               np.reshape(xs, (-1, 1)) + np.reshape(xs, (1, -1))
    else:
        x2 = x2 / lengthscales
        x2s = np.sum(np.square(x2), axis=0)
        return -2 * np.dot(x.T, x2) + \
               np.reshape(xs, (-1, 1)) + np.reshape(x2s, (1, -1))

def euclid_dist(x, x2, lengthscales=None):
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    r2 = square_dist(x, x2, lengthscales)
    return np.sqrt(r2 + 1e-12)

def RBF(x, x2=None, lengthscales=None, kernel_variance=1.):
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((x.shape[1], 0))
    
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    
    return kernel_variance*np.exp(-square_dist(x, x2, lengthscales=lengthscales)/2)

def dRBF(x, x2=None, *args,**kwargs):
    D = x.shape[0]
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((D*x.shape[1], 0))
        
    if x2 is None:
        x2 = x
        
    N_x1 = x.shape[1]
    N_x2 = x2.shape[1]
    
#     # Get kernel matrix (N_x1 x N_x2)
#     if 'Kxx' in kwargs:
#         Kxx = kwargs['Kxx']
#     else:
#         Kxx = RBF(x, x2=x2, *args, **kwargs)
    
#     # Get pairwise distances per columns (D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs = x[:,:,None] - x2[:,:,None].swapaxes(1,2)
    
# #     # Elementwise version for testing
# #     out = np.zeros((D,N_x1,N_x2))
# #     for i in range(N_x1):
# #         for j in range(N_x2):
# #             out[:,i,j] = -(1./np.squeeze(lengthscales**2))*(x[:,i]-x2[:,j]) *Kxx[i,j]
    
#     out = - (1./(np.expand_dims(lengthscales**2,axis=2))) * ( XminusX2_pairwise_diffs * np.expand_dims(Kxx, axis=0) );
#     # We want to stack it to 2D to have shape (D*N_x1,  N_x2)
#     return np.reshape(out, (D*N_x1,N_x2), order='F')

    ### There is some discrepency here between the autograd dRBF and the above manual one, Test later?
    # The jacobian returns a shape
    # (N_x1, N_x2, D, 1)
    jRBF = jacobian(RBF)
    gradRBF = np.concatenate([jRBF(x[:,i:(i+1)], x2, *args, **kwargs) for i in range(N_x1)], axis=0)
    # For every x1 input point, compute a 1xN_x2xD jacobian, then stack them by point getting N_x1, N_x2, D, 1
    
    # We want to stack it to 2D to have shape 
    # (D*N_x1,  N_x2)
    
    # Here the derivative is with respect to the first argument, and it is ANTI-SYMMETRIC (Transpose -> minus sign)
    return np.reshape(gradRBF.swapaxes(1,2).swapaxes(0,1), (D*N_x1, -1), order='F')
    
    
def ddRBF(x, x2=None, *args, **kwargs):
    D = x.shape[0]
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, D*x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((D*x.shape[1], 0))
        
    if x2 is None:
        x2 = x
    
    N_x1 = x.shape[1]
    N_x2 = x2.shape[1]
    
#     # Get kernel matrix (N_x1 x N_x2)
#     if 'Kxx' in kwargs:
#         Kxx = kwargs['Kxx']
#     else:
#         Kxx = RBF(x, x2=x2, *args, **kwargs)
        
#     # Get pairwise distances per columns (D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs = x[:,:,None] - x2[:,:,None].swapaxes(1,2)
    
#     XminusX2_pairwise_diffs_rescaled = (1./(lengthscales**2)[:,:,None])*XminusX2_pairwise_diffs
    
#     # Get the outer product of pairwise distance per instance (D x D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs_rescaled_outer = (np.expand_dims(XminusX2_pairwise_diffs_rescaled,axis=1)
#                                         * np.expand_dims(XminusX2_pairwise_diffs_rescaled,axis=0)
#                                      )
    
#     lengthscales_times_id = np.diag(np.squeeze(1./(lengthscales**2)))[:,:,None,None]
        
    
    
#     out = ((lengthscales_times_id - XminusX2_pairwise_diffs_rescaled_outer) 
#            * np.expand_dims(np.expand_dims(Kxx, axis=0),axis=0)
#            )
    
#     # Out has a shape of D x D x N_x1 x N_x2
#     # We want to stack it to 2D to have shape (D*N_x1, D*N_x2)
#     return np.reshape(out.swapaxes(1,2), (D*N_x1, D*N_x2), order='F')
    
    # The hessian defined here returns a shape
    # (D*N_x1, N_x2, D, 1)
    hRBF = jacobian(dRBF, argnum=1)
    
    hessRBF = np.concatenate([hRBF(x, x2[:,j:(j+1)], *args, **kwargs) for j in range(N_x2)], axis=1)
    
    # We want to stack it to 2D to have shape 
    # (D*N_x1, D*N_x2)
    
    return np.reshape(hessRBF.swapaxes(1,2), (D*N_x1, -1), order='F')


### Expected kernels for noisy input
Compute various kernel expectations of the RBF kernel written above, 
given the first kernel argument is a single Dx1 vector, given by mean $\mu$ (Dx1) and diagonal variance $\sigma$ (Dx1)

Note: Lengthscales are sqrt(gaussian_covariance)

In [19]:
# # Testing noisy (with little noise) vs non-noisy versions
# mu = np.array([[0], [[0]]])
# sigma = np.array([[1e-1], [1e-1]])
# lengthscales = np.array([[1.0], [1.0]])
# kernel_variance = 1.0
# X = np.concatenate([np.arange(-5,5,0.5)[:,None].T, np.arange(-5,5,0.5)[:,None].T])

In [20]:
def RBF_eK(mu, sigma, X, lengthscales=None, kernel_variance=1):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ k(x, X)], a 1 x M array
    """
    if X.shape[1]==0:
        return np.zeros((1, 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
    return RBF(x=mu, 
               x2=X, 
               lengthscales=np.sqrt(lengthscales**2 + sigma), 
               kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod(lengthscales**2 + sigma))
              )

In [21]:
def RBF_exK(mu, sigma, X, lengthscales=None, kernel_variance=1, eK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ x * k(x, X)], a D x M array
    """
    
    if X.shape[1]==0:
        return np.zeros((x.shape[0], 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    if eK is None:
        eK = RBF_eK(mu, sigma, X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    mean_gauss = (X/(lengthscales**2) + mu/sigma)/(1/(lengthscales**2)+(1/sigma))
    
    return eK*mean_gauss

In [22]:
# RBF_exK(np.array([[1.5], [[1]]]), np.array([[1e-20], [1e-2]]), X[:,11:14])

In [23]:
# np.array([[1.5], [[1]]]) * RBF(np.array([[1.5], [[1]]]), X[:,11:14])

In [24]:
def RBF_edK(mu, sigma, X, lengthscales=None, kernel_variance=1, eK=None, exK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ dk(x, X) ], an 1 x (D x M) array
    We want it differentiated with respect to the second argument X -> No minus sign (minus signs cancel)
    """
    if X.shape[1]==0:
        return np.zeros((1, 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    if eK is None:
        eK = RBF_eK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    if exK is None:
        exK = RBF_exK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    
    return np.reshape((exK - X * eK)/(lengthscales**2),(1,-1), order='F')

In [25]:
def RBF_eKK(mu, sigma, X, lengthscales=None, kernel_variance=1):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [k(X, x) * k(x, X) ], an M x M array
    """
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
        
    kXX_scaled = RBF(
                       x=X, 
                       x2=X, 
                       lengthscales=np.sqrt(2*(lengthscales**2)), 
                       kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod(2*(lengthscales**2)))
                    )
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    kXpX_mu = RBF(
                       x=np.reshape(X_pairwise_sums/2,(mu.shape[0], -1), order='F'), 
                       x2=mu, 
                       lengthscales=np.sqrt((lengthscales**2)/2 + sigma), 
                       kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod((lengthscales**2)/2 + sigma))
                    )
    
    out = kXX_scaled * np.reshape(kXpX_mu, (X.shape[1], X.shape[1]), order='F')
    
    # Due to numerical instability, this is not always symmetric, fix:
    out = (out + out.T) / 2.
    
    return out

In [26]:
# # Testing noisy expected kernel outer product vs noiseless 
# RBF_eKK(np.array([[0], [[0]]]), np.array([[1e-13], [1e-13]]), X[:,9:12]) - \
#     RBF(np.array([[0], [[0]]]), X[:,9:12]) * RBF(np.array([[0], [[0]]]), X[:,9:12]).T

In [27]:
def RBF_exKK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [x * k(X, x) * k(x, X) ], a D x M x M array
    """
    if X.shape[1]==0:
        return np.zeros((x.shape[0], 0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    
    # M x M array
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    # D x M x M array
    mean_gauss = ((X_pairwise_sums/2)/(((lengthscales**2)/2)[:,:,None]) + (mu/sigma)[:,:,None])/(
                                                (1/((lengthscales**2)/2)+(1/sigma))[:,:,None])
    
    
    return eKK[:,:,None].swapaxes(1,2).swapaxes(0,1) * mean_gauss

In [28]:
# # Testing noisy expected kernel outer product
# RBF_exKK(np.array([[1], [[1]]]), np.array([[1e-13], [1e-13]]), X[:,9:12])

In [29]:
#     RBF(np.array([[1], [[1]]]), X[:,9:12]) * RBF(np.array([[1], [[1]]]), X[:,9:12]).T

In [30]:
def RBF_exxKK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ (x*x.T) * k(X, x) * k(x, X) ], a D x D x M x M array
    """
    if X.shape[1]==0:
        return np.zeros((x.shape[0], x.shape[0], 0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    
    # M x M array
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    # D x D array
    var_gauss = 1/(1/((lengthscales**2)/2)+(1/sigma))
    
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    # D x M x M array
    mean_gauss = ((X_pairwise_sums/2)/(((lengthscales**2)/2)[:,:,None]) + (mu/sigma)[:,:,None])*(var_gauss[:,:,None])
    
    # D x D x M x M array
    mean_outer = np.expand_dims(mean_gauss, axis=1) * np.expand_dims(mean_gauss, axis=0)
    
    return np.expand_dims(np.expand_dims(eKK, axis=0), axis=0) * (var_gauss[:,:,None,None] + mean_outer)

In [31]:
# # Testing noisy expected kernel outer product
# RBF_exxKK(np.array([[1], [[1]]]), np.array([[1e-2], [1e-2]]), X[:,11:14])[0,1,:,:]

In [32]:
#  RBF(np.array([[1], [[1]]]), X[:,11:14]) * RBF(np.array([[1], [[1]]]), X[:,11:14]).T

In [33]:
def RBF_eKdK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None, exKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [  k(X, x) * dk(x, X)  ], an M x (D x M) array
    """
    
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
        
        
    # m1 x m2
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
        
    # d x m1 x m2
    if exKK is None:
        exKK = RBF_exKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)    
    
    
    # d x m1 x m2,
    # As exKK naturally uses the first argument and
    # X is the second argument in the derivative kernel, we should expand it, such that we iterate along m2 dimension    
    eKdK = (exKK - np.expand_dims(X, axis=1) * np.expand_dims(eKK, axis=0))/((lengthscales**2)[:,:,None])
       
    # We then finally modify the order of axis and the dimensionality to get 
    # the expected m1 - d - m2 order with M x (DM) shape
    
    return np.reshape(eKdK.swapaxes(0,1), (X.shape[1], -1), order='F')

In [34]:
# RBF_eKdK(np.array([[0], [[1]]]), np.array([[1e-20], [1e-20]]), X[:,13:17])[0:2,4:]

In [35]:
#  RBF(np.array([[0], [[1]]]), X[:,13:15]).T * np.reshape(dRBF(X[:,15:17], np.array([[0], [[1]]])),(-1), order='F').T

In [36]:
def RBF_edKdK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None, exKK=None, exxKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [  dk(X, x) * dk(x, X)  ], a (D x M) x (D x M) array
    """
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
            
    # m1 x m2
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    # d1 x m1 x m2
    if exKK is None:
        exKK = RBF_exKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance,
                       eKK = eKK)
    
    # d1 x d2 x m1 x m2
    if exxKK is None:
        exxKK = RBF_exxKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance,
                        eKK = eKK)
    
    
    
    
    edKdK = (exxKK # exKK
        -1.0 * np.expand_dims(np.expand_dims(X, axis=2), axis=1) * np.expand_dims(exKK, axis=0)  # X[:,None,:,None]
        -1.0 * np.expand_dims(np.expand_dims(X, axis=1), axis=0) * np.expand_dims(exKK, axis=1)  # X[None,:,None,:]
        + np.expand_dims(np.expand_dims(X, axis=2), axis=1) * np.expand_dims(np.expand_dims(X, axis=1), axis=0) 
           * np.expand_dims(np.expand_dims(eKK, axis=0), axis=0) # X[:,None,:,None] * X[None,:,None,:] * eKK[None,None,:,:]
    )
    
    # Divide with lengthscales appropriately
    edKdK = edKdK / ((lengthscales.T**2)[:,:,None,None])
    edKdK = edKdK / ((lengthscales**2)[:,:,None,None])
    
    # We then finally modify the order of axis and the dimensionality to get 
    # the expected m1 - d - m2 order with M x (DM) shape
    
    out = np.reshape(edKdK.swapaxes(1,2), (X.shape[0]*X.shape[1], X.shape[0]*X.shape[1]), order='F')
    
    # Due to numerical instability (TODO: Double check if really instab), this is not always symmetric, fix:
    out = (out + out.T) / 2.
    
    return out

In [37]:
# RBF_edKdK(np.array([[0.5], [[0]]]), np.array([[1e-20], [1e-20]]), X[:,11:14])

In [38]:
# np.reshape(dRBF(np.array([[0.5], [[0]]]), X[:,11:14]),(-1,1), order='F') * np.reshape(dRBF(np.array([[0.5], [[0]]]), X[:,11:14]),(-1), order='F').T

In [39]:
# # Testing expected derivative kernel vs derivative kernel in 1D
# xstar = np.arange(-5,5,0.5)

# plots_by_var = []
# for v in [0.01,1.0,2.0,3.0]:
#     plots_by_var.append(
#         plt_type.Scatter(x=np.squeeze(xstar), 
#                       y=np.squeeze(RBF_edK(np.array([[0]]),np.array([[v]]), xstar)), 
#                       mode='markers')
#     )
    

# plots_by_var.append(
#     plt_type.Scatter(x=np.squeeze(xstar), 
#                   y=np.squeeze(dRBF(np.atleast_2d(xstar) , np.array([[0]])).T), 
#                   mode='markers')
#     )
    
# plt(plots_by_var)

# GP prediction given inducing and fixed points

Params:
* Kernel type - RBF (/w dRBF, ddRBF)
* Kernel hyper - variance (eta) and lengthscales
* Inducing point locations (z DxL) and values (u, DxL)
* Fixed point locations (s, DxM) and jacobians (J, MxD_outxD_in)
* Output noise level (sig_eps)

In [40]:
def fp_get_static_K(eta, lengthscales, z, u, s, J, sig_eps, sig_u=None, sig_s=None, sig_J=None):
    """
    Return the cholesky decomposition of the structured kernel matrix, 
    as well as structured targets, 
    so these only have to be computed once per parameter update
    """

    rbf = lambda x1,x2: RBF(x1, x2, lengthscales=lengthscales, kernel_variance=eta)
    drbf = lambda x1,x2: dRBF(x1, x2, lengthscales=lengthscales, kernel_variance=eta)
    ddrbf = lambda x1,x2: ddRBF(x1, x2, lengthscales=lengthscales, kernel_variance=eta)

    ## Diagonal blocks
    # Inducing points
    Ku_u = rbf(z,z)
    if (sig_u is not None):
        Ku_u = Ku_u + np.diag((sig_u*np.ones((Ku_u.shape[0],1))).flatten()) # Noise is due to uncertainty in trans func



    if s.shape[1] > 0:
        # Fixed points
        Ks_s = rbf(s,s)
        if (sig_s is not None):
            Ks_s = Ks_s + np.diag((sig_s*np.ones((Ks_s.shape[0],1))).flatten()) # Noise is due to uncertainty in fix point loc

        # Jacobians at fixed points
        KJ_J = ddrbf(s,s) # Derivatives are not affected by noisy numerics ?        
        if (sig_J is not None):
            KJ_J = KJ_J + np.diag((sig_J*np.ones((KJ_J.shape[0],1))).flatten()) # Noise is due to uncertainty in fix point loc

        ## Off-diagonal blocks
        # Fixed vs inducing
        Ks_u = rbf(s, z)

        # Jac vs inducing
        KJ_u = drbf(s, z)

        # Jac vs fixed
        KJ_s = drbf(s, s)


        ## Stack the matrices appropriately (3 x 3 blocks) # need - signs for derivative transposes?

        K_full = np.concatenate(
                           [np.concatenate([Ku_u, Ks_u, KJ_u]), 
                           np.concatenate([Ks_u.T, Ks_s, KJ_s]), 
                           np.concatenate([KJ_u.T, KJ_s.T, KJ_J])],
                           axis=1)
    else:
        K_full = Ku_u

    #return K_full

    L = None
    # Ensure K_full is positive definite
    while L is None:
        try:
            L = np.linalg.cholesky(K_full)
        except:
            K_full = K_full + 1e-2*np.min(np.linalg.svd(K_full,compute_uv=False))*np.eye(K_full.shape[0])
            L = None
            print("Matrix is not positive definite, adding smallest sv to diagonal")
            set_trace()


#         
# #         # Transform J for appropriate lengthscale (columnwise multiple)
# #         if not (lengthscales is None):
# #             J_scaled = J / np.broadcast_to( (lengthscales[:,:,np.newaxis]).swapaxes(0,1).swapaxes(1,2), J.shape)
# #             print J_scaled
# #         else:
#         J_scaled = J

#         # Reshape J to be appropriate dimensions for the derivative kernel, 
#         # D-dim feature column vectors stacked vertically for each fixed point (DxM) x D output features as number of columns
#         J_scaled = np.reshape(J_scaled.swapaxes(0,1),(-1, J.shape[1]))

    if s.shape[1]>0:
        # J is originally M fixed points x DxD jacobians (Jij is df_i(x)/d_xj, make it into (DinxM) x Dout
        targets = np.concatenate([u.T, s.T, np.reshape(J.swapaxes(1,2),(-1, J.shape[1]), order='C')]) 
        # Sometimes called beta, (L + M + D_inxM) x D_out, where we will use D independent GPs, one for each output dimension 
        # (output and input dimensions are both D)
    else:
        targets = u.T

    params = {'eta': eta, 'lengthscales': lengthscales, 'z': z, 'u': u, 's': s, 'J':J, 'sig_eps': sig_eps, 'sig_u': sig_u}

    return (L, targets, params)
        
def fp_predict(xstar, L, targets, params):
    # Separate predictions per output dimension
    D = targets.shape[1]

    # Compute the kstar kernels:
    Kx_u = RBF(xstar, params['z'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
    Kx_s = RBF(xstar, params['s'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
    Kx_J = dRBF(params['s'], xstar, lengthscales=params['lengthscales'], kernel_variance=params['eta']).T
    Kx_x = RBF(xstar, xstar, lengthscales=params['lengthscales'], kernel_variance=params['eta'])

    Kx_pred = np.concatenate([Kx_u, Kx_s, Kx_J], axis=1).T

    # Compute the predictive mean per dimension then concatenate them
#         Mu_star = np.concatenate([ \
#                 np.dot(Kx_pred.T, 
#                        scipy.linalg.solve_triangular(
#                             L.T, scipy.linalg.solve_triangular(L, targets[:,d:(d+1)], lower=True),
#                             lower=False)
#                        ) \
#              for d in range(D)], axis=1)

#         Mu_star = Mu_star.T

    # Compute the predictive variance
#         Sig_star = Kx_x - np.dot(Kx_pred.T, 
#                                  scipy.linalg.solve_triangular(
#                                         L.T, scipy.linalg.solve_triangular(L, Kx_pred, lower=True),
#                                         lower=False)
#                                  )

    Kinv = np.linalg.inv(np.dot(L,L.T))
    Mu_star = np.atleast_2d(np.squeeze(np.dot(Kx_pred.T, np.dot(Kinv, targets)).T))

    # Get the expected variance at each data point
    Sig_star = Kx_x - np.dot(Kx_pred.T, np.dot(Kinv, Kx_pred))
    Sig_star = np.atleast_1d(np.squeeze(np.diag(Sig_star)))

    # Add the learned transition noise term
    if (params['sig_eps'].size==1):
        Sigma_eps_out = params['sig_eps']*np.eye(D)
    elif (params['sig_eps'].size==D):
        Sigma_eps_out = np.diag(params['sig_eps'].flatten())
    else:
        Sigma_eps_out = params['sig_eps']

    # Add the terms as a 3rd order tensor (NxDxD)
    Sig_star = (np.expand_dims(np.expand_dims(Sig_star, axis=1), axis=2) * np.expand_dims(np.eye(D), axis=0) 
                + np.expand_dims(Sigma_eps_out, axis=0))

    return (Mu_star, Sig_star, Kx_pred)

In [41]:
#?scipy.linalg.solve_triangular

# Filtering of time series

We apply the "direct method" from McHutchon thesis (2014) Chapter 3.6 to timeseries data filtering and parameter estimation via gradient descent

In [42]:
def get_rank1_matrix(A):
    u,s,v = np.linalg.svd(A, full_matrices = False, compute_uv=True)
    return np.atleast_2d(s[0]*np.dot(u[:,0:1], v[0:1,:]))
    

In [43]:
# D = 2
# s = np.zeros((D,0))
# J = np.zeros((0,D,D))

# # Fixed points
# Ks_s = RBF(s,s)

# # Jacobians at fixed points
# KJ_J = ddRBF(s,s) # Derivatives are not affected by noisy numerics ?        

# Ks_s = Ks_s + np.diag((np.ones((Ks_s.shape[0],1))).flatten()) # Noise is due to numerical stability

# # if (sig_J is not None):
# #     KJ_J = KJ_J + np.diag((sig_J*np.ones((KJ_J.shape[0],1))).flatten()) # Noise is due to numerical stability


In [44]:
def update_t_t1(mu_t1_t1, Sigma_t1_t1, L, targets, kernel_variance, Sigma_eps, z, u, lengthscales, s, J):
    """
        Compute mu_t_t1 and Sigma_t_t1 given mu_t1_t1 and Sigma_t1_t1, as well as the parameters 
    """
    
    D = mu_t1_t1.shape[0]
    Nz = z.shape[1]
    Ns = s.shape[1]
    Nzs = Nz+Ns
    NJ = D * s.shape[1]
    
    # K inverse matrix (Nu+Ns+D*Ns)^2 and targets (Nu+Ns+D*Ns)*D scaled
    #Linv = scipy.linalg.solve_triangular(L, np.eye(L.shape[0]), lower=True)
    
    
#     Linv = np.linalg.inv(L) # Numpy inv is by far the fastest for moderate matrices that I expect L to be
#     Kinv_unstab = np.dot(Linv.T, Linv)

    # Numerically stable Kinv using conditional pseudo-inverse
    Kcur = np.dot(L, L.T)
    if np.any(np.isnan(Kcur)):
        set_trace()
    if np.any(np.isinf(Kcur)):
        set_trace()
    svdu, svds, svdv = np.linalg.svd(Kcur, full_matrices = False, compute_uv=True)
    select_evs = ((np.log(svds) - np.max(np.log(svds)))>(-10))
    svdu = svdu[:,select_evs]
    svds = svds[select_evs]
    svdv = svdv[select_evs,:]
    # This happens a lot actually
#     if np.prod(select_evs)==0:
#         print("Warning: Observation covariance is badly conditioned, using reduced rank pseudo-inverse")
        
    # Compute the (possibly pseudo-) inverse
    Kinv = np.dot(np.dot(svdu, np.diag(1./svds)), svdv)
    
#     if not np.allclose(Kinv_unstab, Kinv, atol=1e-8):
#         print("Warning, Kinv is not stable")
#         # set_trace()
    
    alpha = np.dot(Kinv, targets)
    
    
    # Expected kernels 
    X = np.concatenate([z, s],axis=1) # Base observation locations
    dX = s # extra derivative observation locations
    
    eK_zs = RBF_eK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
        
    eKK_zs_zs = RBF_eKK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    exKK_zs_zs = RBF_exKK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=X, 
                         lengthscales=lengthscales, kernel_variance=kernel_variance,
                         eKK = eKK_zs_zs)
    
    # We will only use a part of this, we compute uneccessarily the derivatives at locations z.
    eKdK_zs_zs = RBF_eKdK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=X, 
                          lengthscales=lengthscales, kernel_variance=kernel_variance,
                          eKK = eKK_zs_zs, exKK = exKK_zs_zs)
    
    # Derivatives at fixed points
    if NJ>0:
        edK_s = RBF_edK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=dX, 
                    lengthscales=lengthscales, kernel_variance=kernel_variance,
                    eK=eK_zs[:,-Ns:])

        edKdK_s_s = RBF_edKdK(mu=mu_t1_t1, sigma=Sigma_t1_t1, X=dX, 
                              lengthscales=lengthscales, kernel_variance=kernel_variance,
                              eKK = eKK_zs_zs[-Ns:,-Ns:], exKK = exKK_zs_zs[:,-Ns:,-Ns:])
    
 
    
    # Get blocks
    alpha_zs = alpha[0:Nzs,:]
    Kinv_zs = Kinv[0:Nzs, 0:Nzs]
    
    if NJ>0:
        alpha_J = alpha[-NJ:,:]
        Kinv_J = Kinv[-NJ:, -NJ:]
        Kinv_zs_J = Kinv[0:Nzs, -NJ:]
        eKdK_zs_s = eKdK_zs_zs[:, -NJ:]
    else:
        alpha_J = np.zeros((0,alpha.shape[1]))
        Kinv_J = np.zeros((0,0))
        Kinv_zs_J = np.zeros((Nzs, 0))
        eKdK_zs_s = np.zeros((Nzs, 0))
    
    # Compute the mean
    mu_t_t1 = np.zeros((D,1))
    
    mu_t_t1 = np.dot( eK_zs , alpha_zs)
    if NJ>0:
        mu_t_t1 = mu_t_t1 + np.dot( edK_s , alpha_J)
    
    mu_t_t1 = mu_t_t1.T
    
#     mu_t_t1_noiseless = (
#         np.dot(RBF(mu_t1_t1, X, lengthscales=lengthscales, kernel_variance=kernel_variance), alpha_zs)
#         + np.dot(dRBF(dX, mu_t1_t1, lengthscales=lengthscales, kernel_variance=kernel_variance).T , alpha_J))
    
    # Compute output variances
    
    # Handle the simple (no fixed point) case separately:
    if not NJ>0:
        e_sigma_out = (
            RBF(x=mu_t1_t1, lengthscales=lengthscales, kernel_variance=kernel_variance)
            - np.trace(np.dot(Kinv_zs, eKK_zs_zs))
        )
        
        if e_sigma_out <= 0:
            set_trace()
        
        e_mu_muT_out = np.dot(np.dot(alpha_zs.T, eKK_zs_zs), alpha_zs)
        
    else: # There are fixed points
    
        # Shared for each dimensions (diagonal variances only or every entry??? Yes, expected covariance of indep GPs = 0):
        # Expectation of the variance
        e_sigma_out = (
            RBF(x=mu_t1_t1, lengthscales=lengthscales, kernel_variance=kernel_variance)
            - np.trace(np.dot(Kinv_zs, eKK_zs_zs))
            - 2*np.trace(np.dot(Kinv_zs_J, eKdK_zs_s.T))
            - np.trace(np.dot(Kinv_J, edKdK_s_s))
        )

        # This should also positive, if not, probably numerical error
        if e_sigma_out <= 0:
            e_sigma_out = 0.
    #         print("Matrix inverse is less precise than our estimate of the variance, just set it to eps")
    #         if np.log(np.abs(e_sigma_out))>np.min(np.log(svds) - np.max(np.log(svds))):
    #             # Matrix inverse is less precise than our estimate of the variance, just set it to predef prec value
    #             # print("Matrix inverse is less precise than our estimate of the variance, just set it to eps")
    #             e_sigma_out = 0.
    #         else:
    #             # Something more serious is wrong, debug
    #             set_trace()

        # Variance of the expectation
        # Dimension_specific (D x D matrix)
        e_mu_muT_out = (
            np.dot(np.dot(alpha_zs.T, eKK_zs_zs), alpha_zs)  
            + np.dot(np.dot(alpha_zs.T, eKdK_zs_s), alpha_J)
            + np.dot(np.dot(alpha_zs.T, eKdK_zs_s), alpha_J).T
            + np.dot(np.dot(alpha_J.T, edKdK_s_s), alpha_J)
        )
        
    # End of if (fixed points)
    

    m_mT = np.dot(mu_t_t1, mu_t_t1.T)

    # The variance of the expectation is supposed to be positive semi-definite, delete the negative eigenvalues
    var_e_mu_out = e_mu_muT_out - m_mT
    evs, evecs = np.linalg.eigh(var_e_mu_out)
    var_e_mu_out = np.zeros_like(var_e_mu_out)
    for i1 in range(evs.size):
        if evs[i1]>0:
            var_e_mu_out = var_e_mu_out+ evs[i1]*np.dot(evecs[:,i1][:,None], evecs[:,i1][:,None].T)
#         if evs[i1]==0:
#             print("Zero eigenvalue!")
#         if evs[i1]<0:
#             print("Negative eigenvalue!")

    # The learned transition noise term
    if (Sigma_eps.size==1):
        Sigma_eps_out = Sigma_eps*np.eye(D)
    elif (Sigma_eps.size==D):
        Sigma_eps_out = np.diag(Sigma_eps.flatten())
    else:
        Sigma_eps_out = Sigma_eps
    
    Sigma_t_t1 = e_sigma_out * np.eye(D) + var_e_mu_out + Sigma_eps_out
    
    if not np.allclose(Sigma_t_t1, Sigma_t_t1.T, atol=1e-8):
        set_trace()
        print("Warning: Filtered covariance is not symmetric")
        
#     if np.min(np.linalg.eigvalsh(Sigma_t_t1)) < 0:
#         set_trace()
#         print("Warning: Filtered covariance is not positive definite")
    
    return (mu_t_t1, Sigma_t_t1)
        
    

    
    
    
def update_t_t(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_t):
    """
        Compute mu_t_t and Sigma_t_t given mu_t_t1 and Sigma_t_t1, as well as the parameters 
        
        Computes the negative log marginal likelihood
        - log N(y_t; C*mu_t_t1, C * Sigma_t_t1 * C.T + Sigma_nu)
    """
    
    #set_trace()
    mu_gauss = np.dot(C, mu_t_t1) 
    var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())
    
    # Check for condition number, if high compared to expected precision in y (Sigma_nu), use pseudo-inverse and pseudo-determinant
    prec = 10 #np.min(np.log(1./Sigma_nu))
    # We are doing SVD anyway, use it to compute inverse
    svdu, svds, svdv = np.linalg.svd(var_gauss, full_matrices = False, compute_uv=True)
    # Debug checks
    if not np.allclose(var_gauss, var_gauss.T, atol=1e-8):
        set_trace()
        print("Warning: Observation covariance is not symmetric")
#     evs = np.linalg.eigvals(var_gauss)
#     if np.min(evs)<0:
#         print("Warning: Observation covariance is not positive definite")
    evs = np.log(np.abs(svds))
    if (np.max(evs) - np.min(evs)) > prec:
        # Cut the "too small" eigenvalues, singular values, U columns and V rows 
        # (esentially to perform reduced rank pseudo-inverse and pseudo-determinant, based on expected precision of y)
        select_evs = ((evs - np.max(evs))>(-prec))
        svdu = svdu[:,select_evs]
        svds = svds[select_evs]
        svdv = svdv[select_evs,:]
        evs = evs[select_evs]
        # This happens a lot actually
        # print("Warning: Observation covariance is badly conditioned, using reduced rank pseudo-inverse")
        
    # Compute the (possibly pseudo-) inverse and determinant
    inv_var_gauss = np.dot(np.dot(svdu, np.diag(1./svds)), svdv)
    inv_var_gauss = (inv_var_gauss + inv_var_gauss.T) / 2.
    ldet = np.sum(evs)
    
    
    
    proj_var = np.dot(C, Sigma_t_t1)
    proj_back = np.dot(Sigma_t_t1, np.dot(C.T, inv_var_gauss))
    
    mu_t_t = mu_t_t1 + np.dot(proj_back, (y_t - mu_gauss))
    
    Sigma_t_t = Sigma_t_t1 - np.dot(proj_back, proj_var)
    
    # Also compute negative marginal log likelihood contribution
    D = mu_t_t1.shape[0]*1.0
    
    log_marg_ll = (
        - 1.0* D/2.0*np.log(2*np.pi)
        - 1.0/2.0*ldet
        - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
    )
    
    return (-1.0*log_marg_ll, mu_t_t, Sigma_t_t)
    
    
                                                        
    
    

In [45]:
# # Define the solver algorithm pieces

# def nlog_marg_ll(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_t):
#     """
#         Computes the negative log marginal likelihood
#         - log N(y_t; C*mu_t_t1, C * Sigma_t_t1 * C.T + Sigma_nu)
#     """

#     mu_gauss = np.dot(C, mu_t_t1) 
#     var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())
    
#     # Check for condition number, if high compared to expected precision in y (Sigma_nu), use pseudo-inverse and pseudo-determinant
#     prec = np.mean(np.log(1./Sigma_nu))
#     prec = prec-1
#     # We are doing SVD anyway, use it to compute inverse
#     svdu, svds, svdv = np.linalg.svd(var_gauss, full_matrices = False, compute_uv=True)
#     # Debug checks
#     if not np.allclose(var_gauss, var_gauss.T, atol=1e-8):
#         set_trace()
#         print("Warning: Observation covariance is not symmetric")
# #     evs = np.linalg.eigvals(var_gauss)
# #     if np.min(evs)<0:
# #         print("Warning: Observation covariance is not positive definite")
#     evs = np.log(np.abs(svds))
#     if (np.max(evs) - np.min(evs)) > prec:
#         # Cut the "too small" eigenvalues, singular values, U columns and V rows 
#         # (esentially to perform reduced rank pseudo-inverse and pseudo-determinant, based on expected precision of y)
#         select_evs = ((evs - np.max(evs))>(-prec))
#         svdu = svdu[:,select_evs]
#         svds = svds[select_evs]
#         svdv = svdv[select_evs,:]
#         evs = evs[select_evs]
#         # This happens a lot actually
#         # print("Warning: Observation covariance is badly conditioned, using reduced rank pseudo-inverse")
        
#     # Compute the (possibly pseudo-) inverse and determinant
#     inv_var_gauss = np.dot(np.dot(svdu, np.diag(1./svds)), svdv)
#     inv_var_gauss = (inv_var_gauss + inv_var_gauss.T) / 2.
#     ldet = np.sum(evs)

        
#     # Finish the computation
#     D = mu_t_t1.shape[1]*1.0
    
#     log_marg_ll = (
#         - 1.0* D/2.0*np.log(2*np.pi)
#         - 1.0/2.0*ldet
#         - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
#     )
    
#     return -1.0 * log_marg_ll


# Set up the optimisation algorithm

Initialise parameters, define subsets to optimise, transforms (log space), and priors

In [46]:
# Dy = 2
# T = 7
# Ny = 20
# D = Dy
# Nz = 2
# Ns = 2

# np.random.seed(123)
# y = np.sqrt(0.3)*np.random.randn(Dy,T,Ny)

In [ ]:
# # Load 2D data
# import scipy.io
# from collections import OrderedDict
# data_decision = scipy.io.loadmat('../Matlab/Machens_Brody/MachensBrodySim_20180202T131506_decision.mat', squeeze_me=True)
# simParams = OrderedDict()
# simParams['seed'] = 1234
# simParams['curData'] = "decision"  # decision or loading
# simParams['Ntrain'] = 20
# simParams['delta_t'] = 1
# simParams['T_max'] = min(50, data_decision['y_python'].shape[1])
# simParams['Sigma_y'] = 1e-2
# simParams['rescale'] = 1e3

# curData = data_decision

# x = curData['y_python'][:,range(0,simParams['T_max'],simParams['delta_t']),:simParams['Ntrain']]*simParams['rescale'] # Rescaling to reduce numerical instability
# y = x + np.sqrt(simParams['Sigma_y'])*np.random.randn(*x.shape)
# y.shape

# init_params(y, 2, 30, 3)

In [ ]:
# # Load 1D data
# import scipy.optimize
# import pickle
# import pickle, datetime, time

# curData = pickle.load(open('Experiment_1d_wells_results/well_1d_k2_20180103T181553_ 0_fix_ 20_trials.pkl'))
# y=curData[0]
# init_params(y,1,36,3)

In [ ]:
# D = 2
# Ns = 0
# Nz = 36
    
# Dy = y.shape[0]
    
# # If Dy = D, then just set 
# if D==Dy:
#     C = np.eye(D, Dy)
# else:
#     # Do PCA and choose the first to find initial C (todo)
#     C = np.eye(D, Dy)

# # Estimate the noisyness of y via simple autoregressive AR(1) linear regression predictability, per dim
# y_t = y[:,1:,:]
# y_t1 = y[:,:-1,:]

# y_t_rsh = np.reshape(y_t, (Dy, -1)).T
# y_t1_rsh = np.reshape(y_t1, (Dy, -1)).T

# # Get optimal linear estimate of y_t given the y_t-1 vector    
# y_t_hat = np.dot(y_t1_rsh, np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh))

# # Set the estimated variance of y_t
# Sigma_nu = np.var(y_t_rsh - y_t_hat, axis = 0)[:,None].T
# Sigma_nu = np.reshape(Sigma_nu, (-1,1))

# # Split it into transition and observation noise
# Sigma_eps = Sigma_nu/2.*np.sqrt(2.) # Transition noise
# Sigma_nu = Sigma_nu/2.*np.sqrt(2.) # Observation noise

# # Init GP params
# # First run basic GP regression from y_t-1 -> y_t to estimate kernel hyperparams (skip for now, do simpler things)

# # Set inducing point locations to span y backprojected through Cinv
# if D==Dy:
#     Cinv = np.eye(D, Dy)
# else:
#     Cinv = numpy.linalg.pinv(C)

# Cinvy = np.tensordot(Cinv, y, axes = ([1], [0]))
# grid_min = np.min(Cinvy, axis=(1,2))
# grid_max = np.max(Cinvy, axis=(1,2))

# grid_num = np.floor(Nz ** (1./D)) # Do an evenly spaced D-dim grid using floor(sqrt(Nz))**D points
# remaining_num = Nz - grid_num**D # Initialise the other points randomly

# grid_axes = []
# for d in range(D):
#     grid_axes.append(np.linspace(grid_min[d], grid_max[d], num=grid_num))

# grid_dims = np.meshgrid(*grid_axes)
# grid_dims_flat = []
# for d in range(D):
#     grid_dims_flat.append(grid_dims[d].flatten())
# gridpts = np.array(grid_dims_flat)

# np.random.seed(123)
# if remaining_num > 0:
#     randompts = np.random.rand(D,remaining_num.astype(int))*(grid_max-grid_min)[:,None]+grid_min[:,None]
#     z = np.concatenate([gridpts, randompts], axis=1)
# else:
#     z = gridpts


# # Estimate kernel lengthscale from grid size and number of inducing points
# kernel_variance = np.array([[1.0]])
# lengthscales=(grid_max-grid_min)[:,None]/grid_num * (2.*np.sqrt(D*1.)/2.) # Account for dimension scaling (sqrtD) also
# lengthscales = lengthscales


# # Initialise an empty inducing and fixed point arrays for now
# u = np.zeros(z.shape)
# Sigma_u = np.zeros((z.shape[1],1))
# s = np.zeros((D,0))
# J = np.zeros((0,D,D))
# Sigma_s = np.zeros((0,1))
# Sigma_J = np.zeros((0,1))    

# # Gather backprojected data for AR coefficients
# y_t = Cinvy[:,1:,:]
# y_t1 = Cinvy[:,:-1,:]

# y_t_rsh = np.reshape(y_t, (D, -1))
# y_t1_rsh = np.reshape(y_t1, (D, -1))

# # Run cross-validation on the lengthscales
# test_lengthscales = np.zeros((D, 7))
# for d in range(D):
#     test_lengthscales[d,:] = np.array([1e-1, 5e-1, 1e0, 1.5e0, 2e0, 4e0, 8e0])*lengthscales[d]
# lengthscales_perf = np.zeros((test_lengthscales.shape[1],))
# for cv_folds in range(4):
#     train_inds = np.array(random.sample(range(y_t1_rsh.shape[1]), np.int32(y_t1_rsh.shape[1]*3./4.)))
#     test_inds = np.setdiff1d(np.arange(y_t1_rsh.shape[1]), train_inds)

#     for lengthscale_ind in range(test_lengthscales.shape[1]):
#         # Learn u and Sigma_u from training data
#         L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
#                                      z=y_t1_rsh[:,train_inds], u=y_t_rsh[:,train_inds], 
#                                      s=s, J=J, 
#                                      sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh[:,train_inds].shape[1],1)),
#                                      sig_s=Sigma_s, sig_J=Sigma_J)
#         mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
#         u = mu_star 
#         for i in range(Sigma_u.shape[0]):
#             Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))

#         # Predict RMSE on test data
#         L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
#                                      z=z, u=u, 
#                                      s=s, J=J, 
#                                      sig_eps=Sigma_eps, sig_u = Sigma_u,
#                                      sig_s=Sigma_s, sig_J=Sigma_J)
#         mu_star, sig_star, K_pred = fp_predict(y_t1_rsh[:,test_inds], L, targets, params)
#         lengthscales_perf[lengthscale_ind] = lengthscales_perf[lengthscale_ind] + rmse(y_t_rsh[:,test_inds], mu_star)

# lengthscales = test_lengthscales[:, [np.argmin(lengthscales_perf)]]

# lengthscales_init = lengthscales

# # Initialise inducing point values to GP-autoregression values
# L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales_init, 
#                                      z=y_t1_rsh, u=y_t_rsh, 
#                                      s=s, J=J, 
#                                      sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
#                                      sig_s=Sigma_s, sig_J=Sigma_J)
# mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
# u = mu_star 
# for i in range(Sigma_u.shape[0]):
#     Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))



In [ ]:
# # Initialise inducing point values to GP-autoregression values
# Kt1t1 = RBF(y_t1_rsh, y_t1_rsh, kernel_variance=kernel_variance, lengthscales=lengthscales)
# Kzt1 = RBF(z, y_t1_rsh, kernel_variance=kernel_variance, lengthscales=lengthscales)

# # Do GP regression between the backprojected data points to estimate the values of u
# u = np.random.randn(*z.shape)
# for d1 in range(D):
#     Kpredu = np.dot(Kzt1, np.linalg.inv(Kt1t1+np.diag((Sigma_eps[d1]*np.ones((Kt1t1.shape[0],))).flatten())))
#     u[d1,:] = np.dot(Kpredu, y_t_rsh[d1,:])

# Sigma_u = kernel_variance - np.diag(np.dot(Kpredu, Kzt1.T)) # Uncertainty in inducing points
    
    

In [ ]:
# # Learn a GP, Get probability of every possible fixed point under the GP prior
# res_points = np.int32(2000**(1./D))
# # Get the predictions at xstar values
# gridList = [np.linspace(np.min(z[d,:]),np.max(z[d,:]),res_points) for d in range(D)]
# tmp = np.meshgrid(*gridList)
# xstar = np.concatenate([tmp[d].flatten()[:,None] for d in range(D)], axis=1).T

# s=np.empty((D,0))
# Sigma_s=np.empty((0,1))
# J=np.empty((0,D,D))
# Sigma_J=np.empty((0,1))

# for new_s in range(Ns):
#     L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
#                                          sig_eps=Sigma_eps, sig_s=Sigma_s, sig_J=Sigma_J)
#     mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

#     # Get the marginal probabilities of fixed points given mu_star and sig_star
#     fp_probs = np.zeros((mu_star.shape[1]))
#     s_det_pp_prior = np.ones(*fp_probs.shape)
#     for i in range(mu_star.shape[1]):
#         cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(mu_star[:,i:i+1], sig_star[i,:,:], C, Sigma_nu, xstar[:,i:i+1])
#         fp_probs[i] = np.exp(-1.0*cur_nll_term)
#         # Add a determinental point process prior to the likelihoods
#         Kxs_xs = RBF(np.concatenate([xstar[:,[i]], params['s']], axis=1), 
#                    np.concatenate([xstar[:,[i]], params['s']], axis=1),
#                    lengthscales=params['lengthscales'], kernel_variance=params['eta'])
#         if Kxs_xs.size>0:
#             s_det_pp_prior[i] = np.exp(np.linalg.slogdet(Kxs_xs)[1]*(1.0/(params['s'].shape[1]+1)))
        
    
#     if D == 1:
#         plt([plt_type.Scatter(x=np.squeeze(xstar),y=np.squeeze(fp_probs*s_det_pp_prior))])
#     if D == 2:
#         plt([plt_type.Contour(z=np.reshape(fp_probs*s_det_pp_prior, (res_points,res_points)))])
    
#     # Add the newly defined fixed point
#     f_ind = np.argmax(fp_probs*s_det_pp_prior)
#     s = np.append(s, xstar[:,[f_ind]], axis=1)
#     Sigma_s = np.append(Sigma_s, 
#                         np.atleast_2d(np.mean(np.dot(np.abs(xstar[:,[f_ind]] - mu_star[:,[f_ind]]).T,np.linalg.inv(sig_star[i,:,:])))).T, 
#                         axis=0)    
    
#     # Get predictive information on the derivatives at that fixed point
#     D = targets.shape[1]

#     # Compute the kstar kernels:
#     Kdx_u = dRBF(s[:,-1:], params['z'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
#     Kdx_s = dRBF(s[:,-1:], params['s'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
#     Kdx_J = ddRBF(params['s'], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta']).T
#     Kdx_dx = ddRBF(s[:,-1:], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta'])

#     Kdx_pred = np.concatenate([Kdx_u, Kdx_s, Kdx_J], axis=1).T


#     Kinv = np.linalg.inv(np.dot(L,L.T))
#     Mu_star = np.atleast_2d(np.squeeze(np.dot(Kdx_pred.T, np.dot(Kinv, targets)))).T

#     # Get the expected variance at each data point
#     Sig_star = Kdx_dx - np.dot(Kdx_pred.T, np.dot(Kinv, Kdx_pred))
#     Sig_star = np.atleast_2d(np.squeeze(np.diag(Sig_star))).T

#     # Store them
#     J = np.append(J, np.expand_dims(Mu_star, axis=0), axis=0)
#     Sigma_J = np.append(Sigma_J, Sig_star, axis=0)
    
    

In [ ]:
import random
def rmse(predictions, targets, axis=None):
    return np.sqrt(np.mean(((predictions - targets) ** 2), axis=axis))
def init_params(y, D, Nz, Ns, grad_sigma=True):
    """
    Initialise the model parameters given 
    - y (Dy x T x N): observations
    - D (scalar): latent dimensionality
    - Nz (scalar): number of inducing points
    - Ns (scalar): number of fixed points
    
    Return them as a dictionary
    """    
    
    Dy = y.shape[0]
    
    # If Dy = D, then just set 
    if D==Dy:
        C = np.eye(D, Dy)
    else:
        # Do PCA and choose the first to find initial C (todo)
        #C = np.eye(D, Dy)
        
        # Fix C to the True C for now
        C = np.zeros((Dy, D))
        C[:int(Dy/2),0] = 1.
        C[int(Dy/2):,1] = 1.
        
    # Estimate the noisyness of y via simple autoregressive AR(1) linear regression predictability, per dim
    y_t = y[:,1:,:]
    y_t1 = y[:,:-1,:]

    y_t_rsh = np.reshape(y_t, (Dy, -1)).T
    y_t1_rsh = np.reshape(y_t1, (Dy, -1)).T

    # Get optimal linear estimate of y_t given the y_t-1 vector    
    y_t_hat = np.dot(y_t1_rsh, np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh))

    # Set the estimated variance of y_t
    Sigma_nu = np.var(y_t_rsh - y_t_hat, axis = 0)[:,None].T
    Sigma_nu = np.reshape(Sigma_nu, (-1,1))
    
    
    if D==Dy:
        Cinv = np.eye(D, Dy)
    else:
        Cinv = np.linalg.pinv(C)
    
    # Split it into transition and observation noise
    Sigma_eps = np.dot(Cinv, Sigma_nu)/2.*np.sqrt(2.) # Transition noise
    Sigma_nu = Sigma_nu/2.*np.sqrt(2.) # Observation noise
    
    # Init GP params
    # First run basic GP regression from y_t-1 -> y_t to estimate kernel hyperparams (skip for now, do simpler things)
    
    # Set inducing point locations to span y backprojected through Cinv  
    Cinvy = np.tensordot(Cinv, y, axes = ([1], [0]))
    grid_min = np.min(Cinvy, axis=(1,2))
    grid_max = np.max(Cinvy, axis=(1,2))
    
    grid_num = np.floor(Nz ** (1./D)) # Do an evenly spaced D-dim grid using floor(sqrt(Nz))**D points
    remaining_num = Nz - grid_num**D # Initialise the other points randomly
    
    grid_axes = []
    for d in range(D):
        grid_axes.append(np.linspace(grid_min[d], grid_max[d], num=grid_num))

    grid_dims = np.meshgrid(*grid_axes)
    grid_dims_flat = []
    for d in range(D):
        grid_dims_flat.append(grid_dims[d].flatten())
    gridpts = np.array(grid_dims_flat)
    
    np.random.seed(123)
    if remaining_num > 0:
        randompts = np.random.rand(D,remaining_num.astype(int))*(grid_max-grid_min)[:,None]+grid_min[:,None]
        z = np.concatenate([gridpts, randompts], axis=1)
    else:
        z = gridpts

        
    # Estimate kernel lengthscale from grid size and number of inducing points
    kernel_variance = np.array([[1.0]])
    lengthscales=(grid_max-grid_min)[:,None]/grid_num * (2.*np.sqrt(D*1.)/2.) # Account for dimension scaling (sqrtD) also
    lengthscales = lengthscales
        
        
    # Initialise an empty inducing and fixed point arrays for now
    u = np.zeros(z.shape)
    Sigma_u = np.zeros((z.shape[1],1))
    s = np.zeros((D,0))
    J = np.zeros((0,D,D))
    Sigma_s = np.zeros((0,1))
    Sigma_J = np.zeros((0,1))    
    
    # Gather backprojected data for AR coefficients
    y_t = Cinvy[:,1:,:]
    y_t1 = Cinvy[:,:-1,:]

    y_t_rsh = np.reshape(y_t, (D, -1))
    y_t1_rsh = np.reshape(y_t1, (D, -1))
    
    # Run cross-validation on the lengthscales
    test_lengthscales = np.zeros((D, 7))
    for d in range(D):
        test_lengthscales[d,:] = np.array([1e-1, 5e-1, 1e0, 1.5e0, 2e0, 4e0, 8e0])*lengthscales[d]
    lengthscales_perf = np.zeros((test_lengthscales.shape[1],))
    for cv_folds in range(4):
        train_inds = np.array(random.sample(range(y_t1_rsh.shape[1]), np.int32(y_t1_rsh.shape[1]*3./4.)))
        test_inds = np.setdiff1d(np.arange(y_t1_rsh.shape[1]), train_inds)
        
        for lengthscale_ind in range(test_lengthscales.shape[1]):
            # Learn u and Sigma_u from training data
            L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
                                         z=y_t1_rsh[:,train_inds], u=y_t_rsh[:,train_inds], 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh[:,train_inds].shape[1],1)),
                                         sig_s=Sigma_s, sig_J=Sigma_J)
            mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
            u = mu_star 
            for i in range(Sigma_u.shape[0]):
                Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))
                
            # Predict RMSE on test data
            L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
                                         z=z, u=u, 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = Sigma_u,
                                         sig_s=Sigma_s, sig_J=Sigma_J)
            mu_star, sig_star, K_pred = fp_predict(y_t1_rsh[:,test_inds], L, targets, params)
            lengthscales_perf[lengthscale_ind] = lengthscales_perf[lengthscale_ind] + rmse(y_t_rsh[:,test_inds], mu_star)
            
    lengthscales = test_lengthscales[:, [np.argmin(lengthscales_perf)]]
        
    lengthscales_init = lengthscales
    
    # Initialise inducing point values to GP-autoregression values
    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales_init, 
                                         z=y_t1_rsh, u=y_t_rsh, 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                         sig_s=Sigma_s, sig_J=Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
    u = mu_star 
    for i in range(Sigma_u.shape[0]):
        Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))
    
    if (Ns>0):
        # Learn a GP, Get probability of every possible fixed point under the GP prior
        res_points = np.int32(2000**(1./D))
        # Get the predictions at xstar values
        gridList = [np.linspace(np.min(z[d,:]),np.max(z[d,:]),res_points) for d in range(D)]
        tmp = np.meshgrid(*gridList)
        xstar = np.concatenate([tmp[d].flatten()[:,None] for d in range(D)], axis=1).T

        for new_s in range(Ns):
            L, targets, params = fp_get_static_K(eta=kernel_variance, 
                                                 lengthscales=lengthscales_init, 
                                             z=y_t1_rsh, u=y_t_rsh, 
                                             s=s, J=J, 
                                             sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                             sig_s=Sigma_s, sig_J=Sigma_J)
            mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

            # Get the marginal probabilities of fixed points given mu_star and sig_star
            fp_probs = np.zeros((mu_star.shape[1]))
            s_det_pp_prior = np.ones(*fp_probs.shape)
            for i in range(mu_star.shape[1]):
                cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(
                    mu_star[:,i:i+1], sig_star[i,:,:], C, Sigma_nu, np.dot(C, xstar)[:,i:i+1])
                
                fp_probs[i] = np.exp(-1.0*cur_nll_term)
                # Add a determinental point process prior to the likelihoods
                Kxs_xs = RBF(np.concatenate([xstar[:,[i]], params['s']], axis=1), 
                           np.concatenate([xstar[:,[i]], params['s']], axis=1),
                           lengthscales=params['lengthscales'], kernel_variance=params['eta'])
                if Kxs_xs.size>0:
                    s_det_pp_prior[i] = np.exp(np.linalg.slogdet(Kxs_xs)[1]*(1.0/(params['s'].shape[1]+1)))


            #plt([plt_type.Contour(z=np.reshape(fp_probs*s_det_pp_prior, (res_points,res_points)))])

            # Add the newly defined fixed point
            f_ind = np.argmax(fp_probs*s_det_pp_prior)
            s = np.append(s, xstar[:,[f_ind]], axis=1)
            Sigma_s = np.append(Sigma_s, 
                                np.atleast_2d(np.mean(np.dot(np.abs(xstar[:,[f_ind]] - mu_star[:,[f_ind]]).T,np.linalg.inv(sig_star[f_ind,:,:])))).T, 
                                axis=0)    

            # Get predictive information on the derivatives at that fixed point
            D = targets.shape[1]

            # Compute the kstar kernels:
            Kdx_u = dRBF(s[:,-1:], params['z'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
            Kdx_s = dRBF(s[:,-1:], params['s'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
            Kdx_J = ddRBF(params['s'], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta']).T
            Kdx_dx = ddRBF(s[:,-1:], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta'])

            Kdx_pred = np.concatenate([Kdx_u, Kdx_s, Kdx_J], axis=1).T


            Kinv = np.linalg.inv(np.dot(L,L.T))
            Mu_star = np.atleast_2d(np.squeeze(np.dot(Kdx_pred.T, np.dot(Kinv, targets)))).T

            # Get the expected variance at each data point
            Sig_star = Kdx_dx - np.dot(Kdx_pred.T, np.dot(Kinv, Kdx_pred))
            Sig_star = np.atleast_2d(np.squeeze(np.diag(Sig_star))).T

            # Store them
            J = np.append(J, np.expand_dims(Mu_star, axis=0), axis=0)
            if grad_sigma:
                Sigma_J = np.append(Sigma_J, Sig_star, axis=0) #Real predicted Sigma_J
            else:
                Sigma_J = np.append(Sigma_J, 0.*np.ones(Sig_star.shape), axis=0) #Alternatively use noiseless gradient values for better optimisation
            
        # Reinitialise the u values, taking into account the fixed points we've placed
        L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales_init, 
                                             z=y_t1_rsh, u=y_t_rsh, 
                                             s=s, J=J, 
                                             sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                             sig_s=Sigma_s, sig_J=Sigma_J)
        mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
        u = mu_star
        for i in range(Sigma_u.shape[0]):
            Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))

    
            
#         ###### OLD VERSION
#         ## Initialise fixed point locations from k-means clustering on the back-projected values (k_medians better though)
#         # Transform the data appropriately, and whiten it (kmeans requires) then unwhiten
#         yrsh = (np.reshape(Cinvy, (D,-1)).T)
#         ystd = np.std(yrsh, axis=0,)
#         s = (scipy.cluster.vq.kmeans(yrsh / np.expand_dims(ystd, axis=0), Ns)[0]*np.expand_dims(ystd, axis=0)).T

    
    
    
    #Sigma_u = Sigma_u * 1.5 # To aid further optimization
    
    
    # Init p(x_0) as expected backprojection of first time step
    yrsh = Cinvy[:,0,:]
    mu_0_0 = np.mean(yrsh, axis=1)[:, None]
    Sigma_0_0 = np.var(yrsh, axis=1)[:, None]
#    # Subtract the contribution from Sigma_nu (the linear regression explained observation noise)
#     Sigma_0_0 = 1./(1./Sigma_0_0 - 1./Sigma_nu) # Need more checking
    
    
    paramdict = OrderedDict()
    paramdict['Sigma_eps'] = Sigma_eps
    paramdict['mu_0_0'] = mu_0_0
    paramdict['Sigma_0_0'] = Sigma_0_0
    paramdict['C'] = C
    paramdict['Sigma_nu'] = Sigma_nu
    paramdict['z'] = z
    paramdict['u'] = u
    paramdict['Sigma_u'] = Sigma_u
    paramdict['lengthscales'] = lengthscales
    paramdict['kernel_variance'] = kernel_variance
    paramdict['s'] = s 
    paramdict['J'] = J
    paramdict['Sigma_s'] = Sigma_s
    paramdict['Sigma_J'] = Sigma_J
    
    return paramdict

In [ ]:
import random
def rmse(predictions, targets, axis=None):
    return np.sqrt(np.mean(((predictions - targets) ** 2), axis=axis))
def init_params_lastminute(y, D, Nz, Ns, grad_sigma=True, Sigma_nu_given=None, True_s_given=None):
    """
    Initialise the model parameters given 
    - y (Dy x T x N): observations
    - D (scalar): latent dimensionality
    - Nz (scalar): number of inducing points
    - Ns (scalar): number of fixed points
    
    Return them as a dictionary
    """    
    
    Dy = y.shape[0]
    
    # If Dy = D, then just set 
    if D==Dy:
        C = np.eye(D, Dy)
    else:
        # Do PCA and choose the first to find initial C (todo)
        C = np.eye(D, Dy)
        
    # Estimate the noisyness of y via simple autoregressive AR(1) linear regression predictability, per dim
    y_t = y[:,1:,:]
    y_t1 = y[:,:-1,:]

    y_t_rsh = np.reshape(y_t, (Dy, -1)).T
    y_t1_rsh = np.reshape(y_t1, (Dy, -1)).T

    # Get optimal linear estimate of y_t given the y_t-1 vector    
    y_t_hat = np.dot(y_t1_rsh, np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh))

    # Set the estimated variance of y_t
    Sigma_nu = np.var(y_t_rsh - y_t_hat, axis = 0)[:,None].T
    Sigma_nu = np.reshape(Sigma_nu, (-1,1))
    
    # Split it into transition and observation noise
    if Sigma_nu_given is None:
        Sigma_eps = Sigma_nu/2. # Transition noise
        Sigma_nu = Sigma_nu/2. # Observation noise
    else:
        # Assume observed noise is transition + observation with known observation
        Sigma_eps = Sigma_nu - Sigma_nu_given
        Sigma_nu = Sigma_nu_given
    
    # Init GP params
    # First run basic GP regression from y_t-1 -> y_t to estimate kernel hyperparams (skip for now, do simpler things)
    
    # Set inducing point locations to span y backprojected through Cinv
    if D==Dy:
        Cinv = np.eye(D, Dy)
    else:
        Cinv = numpy.linalg.pinv(C)
        
    Cinvy = np.tensordot(Cinv, y, axes = ([1], [0]))
    grid_min = np.min(Cinvy, axis=(1,2))
    grid_max = np.max(Cinvy, axis=(1,2))
    
    grid_num = np.floor(Nz ** (1./D)) # Do an evenly spaced D-dim grid using floor(sqrt(Nz))**D points
    remaining_num = Nz - grid_num**D # Initialise the other points randomly
    
    grid_axes = []
    for d in range(D):
        grid_axes.append(np.linspace(grid_min[d], grid_max[d], num=grid_num))

    grid_dims = np.meshgrid(*grid_axes)
    grid_dims_flat = []
    for d in range(D):
        grid_dims_flat.append(grid_dims[d].flatten())
    gridpts = np.array(grid_dims_flat)
    
    np.random.seed(123)
    if remaining_num > 0:
        randompts = np.random.rand(D,remaining_num.astype(int))*(grid_max-grid_min)[:,None]+grid_min[:,None]
        z = np.concatenate([gridpts, randompts], axis=1)
    else:
        z = gridpts

        
    # Estimate kernel lengthscale from grid size and number of inducing points
    kernel_variance = np.array([[1.0]])
    lengthscales=(grid_max-grid_min)[:,None]/grid_num * (2.*np.sqrt(D*1.)/2.) # Account for dimension scaling (sqrtD) also
    lengthscales = lengthscales
        
        
    # Initialise an empty inducing and fixed point arrays for now
    u = np.zeros(z.shape)
    Sigma_u = np.zeros((z.shape[1],1))
    s = np.zeros((D,0))
    J = np.zeros((0,D,D))
    Sigma_s = np.zeros((0,1))
    Sigma_J = np.zeros((0,1))    
    
    # Gather backprojected data for AR coefficients
    y_t = Cinvy[:,1:,:]
    y_t1 = Cinvy[:,:-1,:]

    y_t_rsh = np.reshape(y_t, (D, -1))
    y_t1_rsh = np.reshape(y_t1, (D, -1))
    
    # Run cross-validation on the lengthscales
    test_lengthscales = np.zeros((D, 7))
    for d in range(D):
        test_lengthscales[d,:] = np.array([1e-1, 5e-1, 1e0, 1.5e0, 2e0, 4e0, 8e0])*lengthscales[d]
    lengthscales_perf = np.zeros((test_lengthscales.shape[1],))
    for cv_folds in range(4):
        train_inds = np.array(random.sample(range(y_t1_rsh.shape[1]), np.int32(y_t1_rsh.shape[1]*3./4.)))
        test_inds = np.setdiff1d(np.arange(y_t1_rsh.shape[1]), train_inds)
        
        for lengthscale_ind in range(test_lengthscales.shape[1]):
            # Learn u and Sigma_u from training data
            L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
                                         z=y_t1_rsh[:,train_inds], u=y_t_rsh[:,train_inds], 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh[:,train_inds].shape[1],1)),
                                         sig_s=Sigma_s, sig_J=Sigma_J)
#             mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
#             u = mu_star 
#             for i in range(Sigma_u.shape[0]):
#                 Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))
                
#             # Predict RMSE on test data
#             L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=test_lengthscales[:,[lengthscale_ind]], 
#                                          z=z, u=u, 
#                                          s=s, J=J, 
#                                          sig_eps=Sigma_eps, sig_u = Sigma_u,
#                                          sig_s=Sigma_s, sig_J=Sigma_J)

            # Get optimal lengthscale by directly predicting using the training data
            mu_star, sig_star, K_pred = fp_predict(y_t1_rsh[:,test_inds], L, targets, params)
            lengthscales_perf[lengthscale_ind] = lengthscales_perf[lengthscale_ind] + rmse(y_t_rsh[:,test_inds], mu_star)
            
    lengthscales = test_lengthscales[:, [np.argmin(lengthscales_perf)]]
        
    lengthscales_init = lengthscales
    
    # Initialise inducing point values to GP-autoregression values
    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales_init, 
                                         z=y_t1_rsh, u=y_t_rsh, 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                         sig_s=Sigma_s, sig_J=Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
    u = mu_star 
    for i in range(Sigma_u.shape[0]):
        Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))
    
    if (Ns>0):
        # Learn a GP, Get probability of every possible fixed point under the GP prior
        res_points = np.int32(2000**(1./D))
        # Get the predictions at xstar values
        gridList = [np.linspace(np.min(z[d,:]),np.max(z[d,:]),res_points) for d in range(D)]
        tmp = np.meshgrid(*gridList)
        xstar = np.concatenate([tmp[d].flatten()[:,None] for d in range(D)], axis=1).T

        
        L, targets, params = fp_get_static_K(eta=kernel_variance, 
                                         lengthscales=lengthscales_init, 
                                         z=y_t1_rsh, u=y_t_rsh, 
                                         s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                         sig_s=Sigma_s, sig_J=Sigma_J)
        mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)
        for new_s in range(Ns):

            # Get the marginal probabilities of fixed points given mu_star and sig_star
            fp_probs = np.zeros((mu_star.shape[1]))
            s_det_pp_prior = np.ones(*fp_probs.shape)
            for i in range(mu_star.shape[1]):
                cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(mu_star[:,i:i+1], sig_star[i,:,:], C, Sigma_nu, xstar[:,i:i+1])
                fp_probs[i] = np.exp(-1.0*cur_nll_term)
                # Add a determinental point process prior to the likelihoods
                Kxs_xs = RBF(np.concatenate([xstar[:,[i]], params['s']], axis=1), 
                           np.concatenate([xstar[:,[i]], params['s']], axis=1),
                           lengthscales=params['lengthscales'], kernel_variance=params['eta'])
                if Kxs_xs.size>0:
                    s_det_pp_prior[i] = np.exp(np.linalg.slogdet(Kxs_xs)[1]*(1.0/(params['s'].shape[1]+1)))


            #plt([plt_type.Contour(z=np.reshape(fp_probs*s_det_pp_prior, (res_points,res_points)))])

            # Add the newly defined fixed point
            f_ind = np.argmax(fp_probs*s_det_pp_prior)
            
            if True_s_given is not None:
                s = np.append(s, True_s_given[:,[new_s]])
                Sigma_s = np.append(Sigma_s, 1e-12*np.ones((1,1)), axis=0)
            else:
                s = np.append(s, xstar[:,[f_ind]], axis=1)
                Sigma_s = np.append(Sigma_s, 
                                    np.atleast_2d(np.mean(np.dot(np.abs(xstar[:,[f_ind]] - mu_star[:,[f_ind]]).T,np.linalg.inv(sig_star[f_ind,:,:])))).T, 
                                    axis=0)    

            # Get predictive information on the derivatives at that fixed point
            D = targets.shape[1]

            # Compute the kstar kernels:
            Kdx_u = dRBF(s[:,-1:], params['z'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
            Kdx_s = dRBF(s[:,-1:], params['s'], lengthscales=params['lengthscales'], kernel_variance=params['eta'])
            Kdx_J = ddRBF(params['s'], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta']).T
            Kdx_dx = ddRBF(s[:,-1:], s[:,-1:], lengthscales=params['lengthscales'], kernel_variance=params['eta'])

            Kdx_pred = np.concatenate([Kdx_u, Kdx_s, Kdx_J], axis=1).T


            Kinv = np.linalg.inv(np.dot(L,L.T))
            Mu_star = np.atleast_2d(np.squeeze(np.dot(Kdx_pred.T, np.dot(Kinv, targets)))).T

            # Get the expected variance at each data point
            Sig_star = Kdx_dx - np.dot(Kdx_pred.T, np.dot(Kinv, Kdx_pred))
            Sig_star = np.atleast_2d(np.squeeze(np.diag(Sig_star))).T

            # Store them
            J = np.append(J, np.expand_dims(Mu_star, axis=0), axis=0)
            if grad_sigma:
                Sigma_J = np.append(Sigma_J, Sig_star, axis=0) #Real predicted Sigma_J
            else:
                Sigma_J = np.append(Sigma_J, 0.*np.ones(Sig_star.shape), axis=0) #Alternatively use noiseless gradient values for better optimisation
            
        # Reinitialise the u values, taking into account the fixed points we've placed
        L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales_init, 
                                             z=y_t1_rsh, u=y_t_rsh, 
                                             s=s, J=J, 
                                             sig_eps=Sigma_eps, sig_u = np.sqrt(np.sum(Sigma_nu**2)/D)*np.ones((y_t1_rsh.shape[1],1)),
                                             sig_s=Sigma_s, sig_J=Sigma_J)
        mu_star, sig_star, K_pred = fp_predict(z, L, targets, params)
        u = mu_star
        for i in range(Sigma_u.shape[0]):
            Sigma_u[i] = (1./D)*np.sqrt(np.sum(np.diag(sig_star[i,:,:])**2))

    
            
#         ###### OLD VERSION
#         ## Initialise fixed point locations from k-means clustering on the back-projected values (k_medians better though)
#         # Transform the data appropriately, and whiten it (kmeans requires) then unwhiten
#         yrsh = (np.reshape(Cinvy, (D,-1)).T)
#         ystd = np.std(yrsh, axis=0,)
#         s = (scipy.cluster.vq.kmeans(yrsh / np.expand_dims(ystd, axis=0), Ns)[0]*np.expand_dims(ystd, axis=0)).T

    
    
    
    #Sigma_u = Sigma_u * 1.5 # To aid further optimization
    
    
    # Init p(x_0) as expected backprojection of first time step
    yrsh = Cinvy[:,0,:]
    mu_0_0 = np.mean(yrsh, axis=1)[:, None]
    Sigma_0_0 = np.var(yrsh, axis=1)[:, None]
#    # Subtract the contribution from Sigma_nu (the linear regression explained observation noise)
#     Sigma_0_0 = 1./(1./Sigma_0_0 - 1./Sigma_nu) # Need more checking
    
    
    paramdict = OrderedDict()
    paramdict['Sigma_eps'] = Sigma_eps
    paramdict['mu_0_0'] = mu_0_0
    paramdict['Sigma_0_0'] = Sigma_0_0
    paramdict['C'] = C
    paramdict['Sigma_nu'] = Sigma_nu
    paramdict['z'] = z
    paramdict['u'] = u
    paramdict['Sigma_u'] = Sigma_u
    paramdict['lengthscales'] = lengthscales
    paramdict['kernel_variance'] = kernel_variance
    paramdict['s'] = s 
    paramdict['J'] = J
    paramdict['Sigma_s'] = Sigma_s
    paramdict['Sigma_J'] = Sigma_J
    
    return paramdict

In [ ]:
# Define parameter transformations
def transform(paramvec, transform_type, transform_inds=None):
    if transform_inds is None:
        transform_inds = np.arange(len(paramvec))
    
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds,transform_inds)
    extra_params = 0
    
    # Do the transformation from optimised parameter space to actual parameter space
    if transform_type == "Exp":
        out_transformed = np.exp(paramvec[transform_inds])
    elif transform_type == "LogExp":
        out_transformed = np.log1p(np.exp(paramvec[transform_inds]))
    elif transform_type == "Square":
        out_transformed = paramvec[transform_inds]**2
    elif transform_type == "SquareMatrix":
        # Cholesky representation has n_params = (N**2.-N)/2. + N free parameters, 
        # so N**2 + N - 2*n_params=0, so N = 1/2*(-1 + np.sqrt(1+8*n_params))
        # Afterwards we fill in a matrix's lower triangle (in weird autograd compatible way) with our params
        # Then do lowerTriMatrix * lowerTriMatrix.T
        n_params = len(transform_inds)
        N =np.int64(np.round(0.5*(-1 + np.sqrt(1+8*n_params))))
        inds_nonzero = np.ravel_multi_index(np.tril_indices(N),(N,N))
        inds_zero = np.arange(N**2)
        inds_zero = np.setdiff1d(inds_zero,inds_nonzero)
        lowerTriMatrix = np.reshape(
            np.concatenate([paramvec[transform_inds], 
                           np.zeros((N**2-n_params,))]
                          )[np.argsort(np.concatenate([inds_nonzero, inds_zero]))],
                   (N,N))
        out_transformed = np.dot(lowerTriMatrix, lowerTriMatrix.T).flatten()
        extra_params = out_transformed.size - paramvec[transform_inds].size
 
    # Fill in the parameter vector accordingly
    out = np.concatenate([out_transformed, paramvec[inds]])
    
    # Make space for the extra parameters
    inds = np.concatenate([inds[inds<np.min(transform_inds)], inds[inds>np.max(transform_inds)]+extra_params])
    if extra_params > 0:
        transform_inds_out = np.concatenate([transform_inds, np.arange(extra_params)+np.max(transform_inds)])
    elif extra_params < 0:
        transform_inds_out = transform_inds[np.arange(len(transform_inds)+extra_params)]
    else:
        transform_inds_out = transform_inds
        
    # Return the modified parameter vector
    out = out[np.argsort(np.concatenate([transform_inds_out, inds]))]
    
    return (out, transform_inds_out)

def transform_inv(paramvec, transform_type, transform_inds=None):     
    if transform_inds is None:
        transform_inds = np.arange(len(paramvec))
    
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds, transform_inds)
    extra_params = 0
    
    # Do the transformation from optimised parameter space to actual parameter space
    if transform_type == "Exp":
        out_inv_transformed = np.log(paramvec[transform_inds])
    elif transform_type == "LogExp":
        out_inv_transformed = np.log(np.expm1(paramvec[transform_inds]))
    elif transform_type == "Square":
        out_inv_transformed = np.sqrt(paramvec[transform_inds])
    elif transform_type == "SquareMatrix":
        # During the inverse transform, we just have to do Cholesky, then extract the lower triangle
        N = np.int64(np.sqrt(len(transform_inds)))
        fullMatrix = np.reshape(paramvec[transform_inds],(N,-1))
        L = np.linalg.cholesky(fullMatrix)
        out_inv_transformed = L.flatten()[np.ravel_multi_index(np.tril_indices(N),(N,N))]
        extra_params = out_inv_transformed.size - len(transform_inds)
 
    # Fill in the parameter vector accordingly
    out = np.concatenate([out_inv_transformed, paramvec[inds]])
    
    # Make space for the extra parameters
    inds = np.concatenate([inds[inds<np.min(transform_inds)], inds[inds>np.max(transform_inds)]+extra_params])
    if extra_params > 0:
        transform_inds_out = np.concatenate([transform_inds, np.arange(extra_params)+np.max(transform_inds)])
    elif extra_params < 0:
        transform_inds_out = transform_inds[np.arange(len(transform_inds)+extra_params)]
    else:
        transform_inds_out = transform_inds
    
    out = out[np.argsort(np.concatenate([transform_inds_out, inds]))]
    
    return (out, transform_inds_out)

In [ ]:
# Vectorise and transform parameters for optimiser
def params_to_vec(paramdict, transforms={}):
    """
    Return all the parameters vectorised into a flattened vector, but also a dictionary of parameter lengths
    Transforms the parameters to optimiser representation, whose names are in the transforms dictionary
    """
    paramvec = []
    paramdict_ind = OrderedDict()
    paramdict_shape = OrderedDict()
    
    cur_ind = 0
    
    # Add "unexpected" extra parameters via kwargs at the end of the parameter vector 
    for arg_name in paramdict:
        paramdict_shape[arg_name] = paramdict[arg_name].shape
        if np.prod(paramdict_shape[arg_name]) > 0:
            if arg_name in transforms:
                if "inds" in transforms[arg_name]:
                    transform_inds = transforms[arg_name]['inds']
                else:
                    transform_inds = None
                cur_arg = transform_inv(paramdict[arg_name], transforms[arg_name]['type'])[0]
            else:
                cur_arg = paramdict[arg_name]
        else:
            cur_arg = paramdict[arg_name]
            
        paramdict_ind[arg_name] = np.arange(cur_ind, cur_ind+cur_arg.size)
        cur_ind = cur_ind+cur_arg.size
        paramvec.append(np.reshape(cur_arg,(-1,1)))
        
    paramvec = np.concatenate(paramvec, axis=0).flatten()
    
    return (paramvec, paramdict_ind, paramdict_shape)

def vec_to_params(paramvec, paramdict_ind, paramdict_shape, transforms={}):
    """
    Return all the parameters in their original forms as a dictionary
    Transforms the parameters to natural representation, whose names are in the transforms dictionary
    """
    out = OrderedDict()
    
    cur_ind = 0
    for arg_name in paramdict_shape.keys():
        if np.prod(paramdict_shape[arg_name]) > 0:
            if arg_name in transforms:
                if "inds" in transforms[arg_name]:
                    transform_inds = transforms[arg_name]['inds']
                else:
                    transform_inds = None
                cur_arg = transform(paramvec[paramdict_ind[arg_name]], transforms[arg_name]['type'])[0]
            else:
                cur_arg = paramvec[paramdict_ind[arg_name]]
        else:
            cur_arg = np.array([])
        
        out[arg_name] = np.reshape(cur_arg, paramdict_shape[arg_name])
        
    return out

# def params_to_vec(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, **kwargs):
#     """
#     Return all the parameters vectorised into a flattened vector, but also a dictionary of parameter lengths
#     """
#     paramvec = []
#     paramdict_ind = OrderedDict()
#     paramdict_shape = OrderedDict()
    
#     paramdict_shape['Sigma_eps'] = Sigma_eps.shape
#     paramdict_shape['mu_0_0'] = mu_0_0.shape
#     paramdict_shape['Sigma_0_0'] = Sigma_0_0.shape
#     paramdict_shape['C'] = C.shape
#     paramdict_shape['Sigma_nu'] = Sigma_nu.shape
#     paramdict_shape['z'] = z.shape
#     paramdict_shape['u'] = u.shape
#     paramdict_shape['Sigma_u'] = Sigma_u.shape
#     paramdict_shape['lengthscales'] = lengthscales.shape
#     paramdict_shape['kernel_variance'] = kernel_variance.shape
#     paramdict_shape['s'] = s.shape 
#     paramdict_shape['J'] = J.shape
    
#     cur_ind = 0
#     for key in paramdict_shape.keys():
#         paramdict_ind[key] = np.arange(cur_ind, cur_ind+np.prod(paramdict_shape[key]))
#         cur_ind = cur_ind+np.prod(paramdict_shape[key])
#         paramvec.append(np.reshape(eval(key),(-1,1)))
    
#     # Add "unexpected" extra parameters via kwargs at the end of the parameter vector 
#     for arg_name in kwargs:
#         paramdict_shape[arg_name] = kwargs[arg_name].shape
#         paramdict_ind[arg_name] = np.arange(cur_ind, cur_ind+np.prod(paramdict_shape[arg_name]))
#         cur_ind = cur_ind+np.prod(paramdict_shape[arg_name])
#         paramvec.append(np.reshape(kwargs[arg_name],(-1,1)))
        
    
#     paramvec = np.concatenate(paramvec, axis=0).flatten()
    
#     return (paramvec, paramdict_ind, paramdict_shape)

# def vec_to_params(paramvec, paramdict_ind, paramdict_shape):
#     out = []
    
#     cur_ind = 0
#     for key in paramdict_shape.keys():
#         out.append(np.reshape(paramvec[paramdict_ind[key]], paramdict_shape[key]))
        
#     return tuple(out)

In [ ]:
# Add parameter substitution to enable optimising only a subset of params

def replace_params(pvec, opt_params, paramvec):
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds,opt_params)
    out = np.concatenate([pvec, paramvec[inds]])
    out = out[np.argsort(np.concatenate([opt_params, inds]))]
    return out

# def transform(paramvec, transform_type, transform_inds):
#     inds = (np.arange(paramvec.shape[0]))
#     inds = np.setdiff1d(inds,transform_inds)
#     extra_params = 0
    
#     # Do the transformation from optimised parameter space to actual parameter space
#     if transform_type == "Exp":
#         out_transformed = np.exp(paramvec[transform_inds])
#     elif transform_type == "LogExp":
#         out_transformed = np.log1p(np.exp(paramvec[transform_inds]))
#     elif transform_type == "Square":
#         out_transformed = paramvec[transform_inds]**2
#     elif transform_type == "SquareMatrix":
#         # Cholesky representation has n_params = (N**2.-N)/2. + N free parameters, 
#         # so N**2 + N - 2*n_params=0, so N = 1/2*(-1 + np.sqrt(1+8*n_params))
#         # Afterwards we fill in a matrix's lower triangle (in weird autograd compatible way) with our params
#         # Then do lowerTriMatrix * lowerTriMatrix.T
#         n_params = len(transform_inds)
#         N =np.int64(np.round(0.5*(-1 + np.sqrt(1+8*n_params))))
#         inds_nonzero = np.ravel_multi_index(np.tril_indices(N),(N,N))
#         inds_zero = np.arange(N**2)
#         inds_zero = np.setdiff1d(inds_zero,inds_nonzero)
#         lowerTriMatrix = np.reshape(
#             np.concatenate([paramvec[transform_inds], 
#                            np.zeros((N**2-n_params,))]
#                           )[np.argsort(np.concatenate([inds_nonzero, inds_zero]))],
#                    (N,N))
#         out_transformed = np.dot(lowerTriMatrix, lowerTriMatrix.T).flatten()
#         extra_params = out_transformed.size - paramvec[transform_inds].size
 
#     # Fill in the parameter vector accordingly
#     out = np.concatenate([out_transformed, paramvec[inds]])
    
#     # Make space for the extra parameters
#     inds = np.concatenate([inds[inds<np.min(transform_inds)], inds[inds>np.max(transform_inds)]+extra_params])
#     if extra_params > 0:
#         transform_inds_out = np.concatenate([transform_inds, np.arange(extra_params)+np.max(transform_inds)])
#     elif extra_params < 0:
#         transform_inds_out = transform_inds[np.arange(len(transform_inds)+extra_params)]
#     else:
#         transform_inds_out = transform_inds
        
#     # Return the modified parameter vector
#     out = out[np.argsort(np.concatenate([transform_inds_out, inds]))]
    
#     return (out, transform_inds_out)

# def transform_inv(paramvec, transform_type, transform_inds):        
#     inds = (np.arange(paramvec.shape[0]))
#     inds = np.setdiff1d(inds, transform_inds)
#     extra_params = 0
    
#     # Do the transformation from optimised parameter space to actual parameter space
#     if transform_type == "Exp":
#         out_inv_transformed = np.log(paramvec[transform_inds])
#     elif transform_type == "LogExp":
#         out_inv_transformed = np.log(np.expm1(paramvec[transform_inds]))
#     elif transform_type == "Square":
#         out_inv_transformed = np.sqrt(paramvec[transform_inds])
#     elif transform_type == "SquareMatrix":
#         # During the inverse transform, we just have to do Cholesky, then extract the lower triangle
#         N = np.int64(np.sqrt(len(transform_inds)))
#         fullMatrix = np.reshape(paramvec[transform_inds],(N,-1))
#         L = np.linalg.cholesky(fullMatrix)
#         out_inv_transformed = L.flatten()[np.ravel_multi_index(np.tril_indices(N),(N,N))]
#         extra_params = out_inv_transformed.size - len(transform_inds)
 
#     # Fill in the parameter vector accordingly
#     out = np.concatenate([out_inv_transformed, paramvec[inds]])
    
#     # Make space for the extra parameters
#     inds = np.concatenate([inds[inds<np.min(transform_inds)], inds[inds>np.max(transform_inds)]+extra_params])
#     if extra_params > 0:
#         transform_inds_out = np.concatenate([transform_inds, np.arange(extra_params)+np.max(transform_inds)])
#     elif extra_params < 0:
#         transform_inds_out = transform_inds[np.arange(len(transform_inds)+extra_params)]
#     else:
#         transform_inds_out = transform_inds
    
#     out = out[np.argsort(np.concatenate([transform_inds_out, inds]))]
    
#     return (out, transform_inds_out)

# def do_inv_transforms(paramvec, transforms):
#     # Change from "natural" parameter space into the "optimised" parameter space
#     for tf in transforms:
#         paramvec = transform_inv(paramvec, tf['type'], tf['inds'])[0]
        
#     return paramvec

# def do_transforms(paramvec, transforms):
#     # Change from "optimised" parameter space into the "natural" parameter space
#     # Transforms have to be done in reverse order than the transform_inds were determined during the inverse transforms
#     for tf in reversed(transforms):
#         paramvec = transform(paramvec, tf['type'], tf['inds'])[0]
    
#     return paramvec

In [ ]:
# Add prior types and derivatives and compute contribution
# from autograd.extend import primitive, defvjp

def add_prior(paramdict, prior_type, prior_metadata={}):
    """
    Compute a the negative LL of the current paramdict, under a prior of given type and metadata,
    """
    if prior_type=="LogGamma":
        """
            Gamma distribution over log(x) with parameters [shape, scale, log10loc]
        """
        prior_params = prior_metadata['hyperparams']
        out = 0
        for key in prior_metadata['names']:
            for x in paramdict[key].flatten():
                shape = prior_params[0]
                scale = prior_params[1]
                log10loc = prior_params[2]

                # Rescale and shift x
                x = (np.log10(x)-log10loc)/scale
                
                # Compute logpdf of standardised gamma (scale = 1, loc =0)            
                cur_out = (shape-1.0) *np.log(x) - x - scipy.special.gammaln(shape)
                
                # Compute the negative logpdf for the scaled variable
                cur_out = -1.*(cur_out - scale)
                
                # Add to running sum
                out = out + cur_out
                
    elif prior_type=="Exponential":
        """
            Exponential distribution over x with parameters [scale]
        """
        prior_params = prior_metadata['hyperparams']
        out = 0
        for key in prior_metadata['names']:
            for x in paramdict[key].flatten():
                scale = prior_params[0]

                # Compute the negative logpdf for the scaled variable
                cur_out = -1.*(-x/scale - scale)
                
                # Add to running sum
                out = out + cur_out
        
    elif prior_type=="InducingDPP":
        """
        Assume x is built from parts: [u,z,Sigma_u, kernelparams] = "partition"(x)
        Places a prior on the values u, such that if they are "close" in location z 
        (with closeness defined by a kernel in metadata['kernel_func'](z,z | kernelparams) ),
        then the values of u should be close as well:
        P(z) = DeterminentalPointProcessPrior
        """
        
        [u,z,Sigma_u,kernelparams] = prior_metadata['unpack_dict'](paramdict)
        Kzz = prior_metadata['kernel_func'](z,z, **kernelparams)
        curcovm = Kzz+np.diag(Sigma_u.flatten())
        curcovm_inv = np.linalg.inv(curcovm)
        ldet = np.linalg.slogdet(curcovm)[1]
        out = - ldet*(1.0/np.max([1.0, z.shape[1]]))*prior_metadata['prior_weight']
            
    
    elif prior_type=="InducingSmooth":
        """
        Assume x is built from parts: [u,z,Sigma_u, kernelparams] = metadata['unpack_x'](x)
        Places a prior on the values u, such that if they are "close" in location z 
        (with closeness defined by a kernel in metadata['kernel_func'](z,z | kernelparams) ),
        then the values of u should be close as well:
        P(u | z, Sigma_u, kernelparams) = Normal( 0, K_zz + diag(Sigma_u) )
        """

        [u,z,Sigma_u,kernelparams] = prior_metadata['unpack_dict'](paramdict)

        Kzz = prior_metadata['kernel_func'](z,z, **kernelparams)

        out = 0
        try:
            curcovm = Kzz+np.diag(Sigma_u.flatten())
            curcovm_inv = np.linalg.inv(curcovm)
            ldet = np.linalg.slogdet(curcovm)[1]
            for d1 in range(u.shape[0]):
                curlogpdf = - 1.0/2.0*ldet - 1.0/2.0*np.dot(np.dot(u[d1:(d1+1),:], curcovm_inv), u[d1:(d1+1),:].T)
                out = out - curlogpdf*prior_metadata['prior_weight']

        except:
            set_trace()
            
    elif prior_type=="InducingSmooth_and_DPP":
        """
        Combining smoothness and DPP priors to avoid unnecessary computations
        """

        [u,z,Sigma_u,kernelparams] = prior_metadata['unpack_dict'](paramdict)

        Kzz = prior_metadata['kernel_func'](z,z, **kernelparams)

        out = 0
        try:
            curcovm = Kzz+np.diag(Sigma_u.flatten())
            curcovm_inv = np.linalg.inv(curcovm)
            ldet = np.linalg.slogdet(curcovm)[1]
            for d1 in range(u.shape[0]):
                curlogpdf = - 1.0/2.0*ldet - 1.0/2.0*np.dot(np.dot(u[d1:(d1+1),:], curcovm_inv), u[d1:(d1+1),:].T)
                out = out - curlogpdf*prior_metadata['prior_weight_Smooth']

            out = out - ldet*(1.0/np.max([1.0, z.shape[1]]))*prior_metadata['prior_weight_DPP']
        except:
            set_trace()
        
    else:
        out = 0
        
    return out
            
    
# # # Examples

# # # LogGamma prior
# pf = create_prior("LogGamma", [2., 1.5, -6.])
# x = np.logspace(-6,2,100)
# plt(plt_type.Figure(data=[plt_type.Scatter(x=x, y=np.exp(-pf(x)))], layout=plt_type.Layout(xaxis=dict(type= "log"))))
# plt(plt_type.Figure(data=[plt_type.Scatter(x=x, y=pf(x))], layout=plt_type.Layout(xaxis=dict(type= "log"))))
            

# Run optimisation algorithm

See McHutchon thesis 3.6 Direct method, Algorithm 1

In [ ]:
# Define a single complete iteration for vectorised parameter collection
def time_full_iter(paramvec, y, paramdict_ind, paramdict_shape, 
                   transforms={}, priors={}, ret_smoothed = False, cutoff = None):
    """
    Run a full iteration filtering forward in time, returning the total negative log likelihood (the objective)
    as a function of a vectorised collection of all our parameters, so we can take easy gradient steps
    """
    
    # The objective is to minimise the negative log likelihood
    neg_log_likelihood = 0
    
    Dy = y.shape[0]
    T = y.shape[1]
    Ny = y.shape[2]
        
    paramvec_orig = paramvec
    
    # Get the original representation of parameters
    paramdict = vec_to_params(paramvec, paramdict_ind, paramdict_shape, transforms)
    
    # Compute contributions of the priors if they exist
    nprior_contrib = 0
    for prior in priors:
        nprior_contrib = nprior_contrib + add_prior(paramdict, prior['type'], prior['metadata'])
    
    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        list(paramdict.values())[:12]
    
    if np.any(np.isnan(lengthscales)):
        set_trace()
    
    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']
        
    D = mu_0_0.shape[0]
    
    # Collect smoothed latent trajectories
    x_all_t1 = np.zeros((D, T, Ny))
    x_all_t = np.zeros((D, T, Ny))
    sig_all_t1 = np.zeros((D*D, T, Ny))
    sig_all_t = np.zeros((D, T, Ny))
    neg_log_likelihood_all = np.zeros((T, Ny))
    

    
    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = Sigma_u, sig_s=Sigma_s, sig_J=Sigma_J)

    for n in range(Ny):
        
        mu_t1_t1 = mu_0_0
        Sigma_t1_t1 = Sigma_0_0
        
        for t in range(T):
            mu_t_t1, Sigma_t_t1 = update_t_t1(mu_t1_t1, Sigma_t1_t1, L, targets, kernel_variance, 
                                              Sigma_eps, z, u, lengthscales, s, J)

#             # Check "condition number" and add diag term to correct if needed
#             if (np.min(np.diag(Sigma_t_t1))<1e-6):
#                 Sigma_t_t1 = Sigma_t_t1 + 1e-6*np.eye(Sigma_t_t1.shape[0])
            
            if ret_smoothed:
                x_all_t1[:,t,n] = mu_t_t1.flatten()            
                sig_all_t1[:,t,n] = np.reshape(Sigma_t_t1,(1,-1))
            
            # cur_nll_term = nlog_marg_ll(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y[:,t:(t+1),n])
            

            cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y[:,t:(t+1),n])
            neg_log_likelihood = neg_log_likelihood + cur_nll_term
                        
            Sigma_t1_t1 = np.diag(Sigma_t1_t1)[:,None]
            
            if cutoff is not None:
                if t>cutoff:
                    # Integrate the first Tau data points, but then no more data available (ignore mu_t1_t1 and Sigma_t1_t1)
                    mu_t1_t1 = mu_t_t1
                    Sigma_t1_t1 = np.diag(Sigma_t_t1)[:,None]
            
            # Check "condition number" and add diag term to correct if needed            
            if (np.min(Sigma_t1_t1)<1e-6):
                #set_trace()
                # Sigma_t1_t1 = Sigma_nu # This doesn't work where Dx != Dy
                Sigma_t1_t1 = Sigma_t1_t1 + 1e-6
                # print("Warning, Sigma_t1_t1 is not pos def, resetting it")
                
            
            
            if ret_smoothed:
                x_all_t[:,t,n] = mu_t1_t1.flatten()
                sig_all_t[:,t,n] = np.diag(Sigma_t1_t1).flatten()
                neg_log_likelihood_all[t,n] = cur_nll_term

            #print(np.concatenate([y[:,t:(t+1),0], mu_t_t1, mu_t1_t1], axis=1))
            
    return (neg_log_likelihood + nprior_contrib, 
            x_all_t1, x_all_t, sig_all_t1, 
            sig_all_t, neg_log_likelihood_all, nprior_contrib)
    

In [ ]:
def minibatch_iter(paramvec, y, *args, **kwargs):
    if "minibatch_func" in kwargs:
        minibatch_func = kwargs.pop("minibatch_func")
    elif "minibatch_size" in kwargs:
        minibatch_size = kwargs.pop("minibatch_size")
        def minibatch_func(data):
            return data[:,:,random.sample(range(data.shape[2]),minibatch_size)]
    else:
        def minibatch_func(data):
            return data
    
    return time_full_iter(paramvec, minibatch_func(y), *args, **kwargs)

In [ ]:
# # Run the algorithm

# Dy = 2
# T = 7
# Ny = 20
# D = Dy
# Nz = 2
# Ns = 2

# np.random.seed(123)
# y = 0.3*np.random.randn(Dy,T,Ny)+0.5

# (mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J) = \
#     init_params(y, D, Nz, Ns)


# paramvec = params_to_vec(mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J)
    
# # Optimise only certain elements of paramvec
# cur_pvec = paramvec[0:2]
# objective_with_grad = value_and_grad(lambda pvec: 
#                                          time_full_iter(np.concatenate([pvec, paramvec[2:]]), 
#                                                         y, D, Nz, Ns), argnum=0)


# result = scipy.optimize.minimize(objective_with_grad, cur_pvec, jac=True, method='CG',
#                       options={'maxiter':3, 'disp':True})

# # 
# # for it in range(1):
# #     a = objective_with_grad(cur_pvec)
# #     b = elem_grad(cur_pvec)
    
# #     print np.concatenate([a[1], b], axis=1)
    
# #     print a[0]
# #     cur_pvec = cur_pvec - 1e-5*a[1]




## Custom Optimizers

1. ADAM optimizer (Kingma 2014)

In [ ]:
import os, time
def adamOptimize(fval, fgrad, theta, options={},
                 callback_func=None, callback_params=[]):
    
    default_options = {
        'alpha':1e-3,
        'beta1':0.9,
        'beta2':0.999,
        'epsilon':1e-8,
        'maxiter':1000,
        'ma_iters':20,
        'obj_eval_iters':1 # Evaluate the objective every this many iterations
    }
    
    for key in default_options:
        if key not in options:
            options[key] = default_options[key]
    
    alpha=options['alpha']
    beta1=options['beta1']
    beta2=options['beta2']
    epsilon=options['epsilon']
    maxiter=options['maxiter']
    
    t = 0
    
    # Create same optimizeResult object as scipy
    result = scipy.optimize.OptimizeResult()
    
    # Store information for subsequent diagnostics
    init_time = time.time()
    all_t = []
    all_theta = []
    all_times = []
    all_objs = [] # Iter and obj value
    
    # Compute the function to get the initial objective and gradient
    converged = False
    message = "Not converged"
    
    # Store a moving average of the objective as a stopping condition
    obj_ma = list((np.inf,)*options['ma_iters'])
    obj_new = np.inf
    
    while (not converged) and (t < options['maxiter']):
        
        all_t.append(t)
        all_theta.append(theta)
        all_times.append(time.time()-init_time)
        
        
        if not np.mod(t, options['obj_eval_iters']):
            obj_new = fval(theta) # This may be on full data
        gr = fgrad(theta) # This may be on minibatch
        if t == 0:
            m = np.zeros(*gr.shape)
            v = np.zeros(*gr.shape)            
            obj_orig = obj_new # This may be on full data
            obj = obj_orig
            
        t = t+1
        
        m = beta1*m + (1-beta1)*gr
        v = beta2*v + (1-beta2)*(gr**2)
        
        mhat = m/(1-beta1**t)
        vhat = v/(1-beta2**t)
        theta_new = theta - alpha*mhat/(np.sqrt(vhat)+epsilon)
        
        # Check for convergence in theta
        if (np.sqrt(np.sum((theta_new-theta)**2))<1e-6*np.sqrt(np.sum(theta**2))):
            converged = True
            message = "Theta converged"
            
        # Objective convergence criterion (based on moving average of true objective)
        if np.mean(np.array(obj_ma))<obj_new:
            converged = True
            message = "Objective converged"
        else:        
            theta = theta_new
            obj = obj_new
            obj_ma.pop(0)
            obj_ma.append(obj_new)
            all_objs.append(np.array([t-1, obj])) # Iter and obj value
        
        if (t < 11) or (not np.mod(t-1,10)):
#             obj = fval(theta)
#             all_objs.append(np.array([t-1, obj])) # Iter and obj value
            print_str = "Iter %3d     Objective: %6f      |grad| %4f\n" % (t-1, obj, np.sqrt(np.sum((mhat/(np.sqrt(vhat)+epsilon))**2)))
            os.write(1, print_str.encode())
                
        result.x = theta
        result.fun = obj
        result.converged = converged
        result.iters = all_t
        result.objs = all_objs
        result.theta_hist = all_theta
        result.times = all_times
        result.options = options
        result.message = message
    
        if callback_func is not None:
            callback_func(result, *callback_params)
    
    
    
    
    return result
    
    
    

# Evaluation metrics

This section implements the following evaluation metrics for a fit on training data, given held out test data

1. One time point ahead prediction - RMSE and likelihood of observed data under GP posterior
2. Full trace prediction - (weighted) RMSE and likelihood of observed data under GP posterior



In [ ]:
def pred_lin_AR1(y_test, y, cutoff=None):
    # Perform linear AR(1) on y as baseline
    Dy = y_test.shape[0]
    y_t = y[:,1:,:]
    y_t1 = y[:,:-1,:]

    y_t_rsh = np.reshape(y_t, (y.shape[0], -1)).T
    y_t1_rsh = np.reshape(y_t1, (y.shape[0], -1)).T

    # Get least squares linear regression weights from original dataset
    weights_AR = np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh)

    # Get optimal linear estimate of y_tt given the y_t-1 vector
    if cutoff is None:
        y_test_t1_rsh = np.reshape(y_test[:,:-1,:], (y.shape[0], -1)).T
        y_test_t_hat = np.dot(y_test_t1_rsh, weights_AR)
        y_test_t_hat = np.reshape(y_test_t_hat.T, (y_test.shape[0], y_test.shape[1]-1, y_test.shape[2]))
    else:
        # Until cutoff do the same
        y_test_t1_rsh = y_test[:,0:min(cutoff+1,y_test.shape[1]-1),:]
        y_test_t_hat = np.einsum('ed,dtn->etn', weights_AR, y_test_t1_rsh)
        # From cutoff do successive one step ahead predicitons
        for t in range(cutoff+1, y_test.shape[1]-1):
            y_test_t_hat = np.concatenate(
                [y_test_t_hat, 
                 np.einsum('ed,dtn->etn', weights_AR, y_test_t_hat[:,-1:,:])],
                axis = 1)
            
    
    # Set first time point prediction as the mean of observed first time points
    y_test_t_hat0 = np.tile(np.mean(y[:,0:1,:],axis=2),[1,1,y_test.shape[2]])
    
    # Concatenate
    y_test_t_hat = np.concatenate([y_test_t_hat0, y_test_t_hat], axis=1)

    return y_test_t_hat

In [ ]:
# final_paramvec = log_transform_inv(replace_params(all_results[-1].x, opt_params, init_paramvec), log_transformed)

def pred_GP(y_test, final_paramvec, transforms, dict_ind, dict_shape, priors={}, cutoff = None):
    """
    Returns a list, with elements:
    0 - Objective function value (nll + nlogprior) [scalar]
    1 - Latent mean at t, given data up to t-1 [DxTxN]
    2 - Latent mean at t, given data up to t [DxTxN]
    3 - Latent covariance at t, given data up to t-1 (square matrix) [(D*D)xTxN]
    4 - Latent covariance at t, given data up to t (vector of the diagonal) [DxTxN]
    5 - Negative log likelihood of each data point [TxN]
    6 - The negative log likelihood of the paramaters under the priors [scalar]
    7 - The mean of the prediction in observed data space [DyxTxN]
    """
    #     time_full_iter returns these:
    #     (neg_log_likelihood + nprior_contrib, 
    #             x_all_t1, x_all_t, sig_all_t1, 
    #             sig_all_t, neg_log_likelihood_all, nprior_contrib)
    
    results_GP = time_full_iter(final_paramvec, 
                          y_test, dict_ind, dict_shape, transforms=transforms, priors=priors,
                          ret_smoothed=True, cutoff = cutoff)
    
    # Get the y mean predictions via C' * x_t1
    y_test_pred = np.einsum('ed,dtn->etn', np.reshape(final_paramvec[dict_ind['C']], dict_shape['C']), results_GP[1])
    
    results_GP = list(results_GP)
    results_GP.append(y_test_pred)
    
    return results_GP

In [ ]:
def rmse(predictions, targets, axis=None):
    return np.sqrt(np.mean(((predictions - targets) ** 2), axis=axis))

# Testing

In [ ]:
# # Training data in 2D, /w derivative observations
# # z = np.array([[-2, 0, 2], [1, 0, 1]])
# # u = np.array([[-1, 1, -3], [-1, 1, -3]])
# z = np.array([[-3.0], [-3.0]])
# u = np.array([[2.0], [1.0]])
# s = np.array([[-1.0, 2.0], 
#               [-1.0, 3.0]])
# # Set the derivates to be N fixed point x DxD Jacobians
# J = np.concatenate(
#         [np.array([[[-1, 0], 
#                    [0, -1]]]),
#          np.array([[[2, -1], 
#                    [0, 0]]])
#          ],
#     axis = 0)*1.0

# which_fixed_point = 1
# xtmp, ytmp = np.meshgrid(np.concatenate([np.arange(-5,5,0.5), np.arange(s[0,which_fixed_point]-0.1,s[0,which_fixed_point]+0.1,0.01)]),
#                          np.concatenate([np.arange(-5,5,0.5), np.arange(s[1,which_fixed_point]-0.1,s[1,which_fixed_point]+0.1,0.01)]))
# # xtmp, ytmp = np.meshgrid(np.concatenate([np.arange(s[0,which_fixed_point]-0.1,s[0,which_fixed_point]+0.1,0.01)]),
# #                          np.concatenate([np.arange(s[1,which_fixed_point]-0.1,s[1,which_fixed_point]+0.1,0.01)]))
# # xtmp, ytmp = np.meshgrid(np.arange(1.9,2.1,0.005),
# #                          np.arange(2.9,3.1,0.005))

# xstar = np.concatenate([xtmp.flatten()[:,None], ytmp.flatten()[:,None]], axis=1).T


# # # Training data in 1D, /w derivative observations
# # z = np.array([[1.0]])
# # u = np.array([[1.0]])

# # s = np.array([[0.0, 2.0]])
# # J = np.array([[[-0.5, 0.5]]])

# # xstar = np.array([np.concatenate([np.arange(-5,5,0.5), 
# #                                   np.arange(s[0,0]-0.1,s[0,0]+0.1,0.01), 
# #                                   np.arange(s[0,1]-0.1,s[0,1]+0.1,0.01)])])



In [ ]:
# # Create the GP functions
# fp_get_static_K, fp_predict = create_fp_gp_funcs()

In [ ]:
# # Compute the params
# L, targets, params = fp_get_static_K(eta=1.0, lengthscales=1.5*np.ones((z.shape[0], 1)), z=z, u=u, s=s, J=J, sig_eps=1e-1)
# #For debug
# #K_full = fp_get_static_K(eta=1.0, lengthscales=None, z=z, u=u, s=s, J=J, sig_eps=1e-1)
        
# mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)
# #mu_star, sig_star, K_pred = fp_predict(np.array([[0,1]]), L, targets, params)

In [ ]:
# plt([plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star), mode='markers'),
#      plt_type.Scatter(x=np.squeeze(s), y=np.squeeze(s), mode='markers') \
#     ])


In [ ]:
# plt([plt_type.Scatter3d(x=np.squeeze(xstar[0,:]), 
#                         y=np.squeeze(xstar[1,:]), 
#                         z=np.squeeze(mu_star[:,1]), mode='markers')])

In [ ]:
# np.set_printoptions(precision=3)

In [ ]:
# # 2D input

# [mu_t_t1, Sigma_t_t1] = update_t_t1(np.array([[1.0], [[0.5]]]),np.array([[1e-2], [1e-2]]), 
#             L, targets, 
#             params["eta"], params["z"], params["u"], params["lengthscales"],
#             params["s"], params["J"])

# print([mu_t_t1, Sigma_t_t1])

# [mu_star, sig_star, Kx_pred] = fp_predict(np.array([[1.0], [[0.5]]]), L, targets, params)

# print([mu_star, sig_star])

# # # 1 D input

# # [mu_t_t1, Sigma_t_t1] = update_t_t1(np.array([[-2.0]]),np.array([[1e-2]]), 
# #             L, targets, 
# #             [], params["eta"], params["z"], params["u"], params["sig_eps"], params["lengthscales"],
# #             params["s"], params["J"])

# # print([mu_t_t1, Sigma_t_t1])

# # [mu_star, sig_star, Kx_pred] = fp_predict(np.array([[-2.0]]), L, targets, params)

# # print([mu_star, sig_star])


# Generate example data

## 1D,  1 well example

1 D transition function, static plus noise, except for pull towards +1.32

In [ ]:
# def draw_trial(well_loc, T, Sigma_eps, Sigma_nu, mu_0_0, Sigma_0_0, well_width):
#     x0 = mu_0_0 + np.sqrt(Sigma_0_0)*np.random.randn(1)
#     x = np.zeros((1,T))
#     y = np.zeros((1,T))
#     for t in range(T):
#         if t==0:
#             xprev = x0
#         else:
#             xprev = x[:,t-1]

#         if ((xprev<(well_loc - np.sqrt(Sigma_eps)*well_width)) or (xprev>(well_loc + np.sqrt(Sigma_eps)*well_width))):
#             x[:,t] = xprev + np.sqrt(Sigma_eps)*np.random.randn(1)
#         else:
#             x[:,t] = well_loc + np.sqrt(Sigma_eps)*np.random.randn(1)
        
#         y[:,t] = x[:,t] + np.sqrt(Sigma_nu)*np.random.randn(1)
        
#     return (x,y)    

In [ ]:
# Ny = 13
# well_loc = 1.41
# T = 10
# Sigma_eps = 1e-1
# Sigma_nu = 1e-1
# mu_0_0 = 0.7
# Sigma_0_0 = 1e-1
# well_width = 3

# # Collect trials into runnable object
# all_trials_x = []
# all_trials_y = []
# for n in range(Ny):
#     x,y = draw_trial(well_loc, T, Sigma_eps, Sigma_nu, mu_0_0, Sigma_0_0, well_width)
    
#     all_trials_x.append(x[:,:,None])
#     all_trials_y.append(y[:,:,None])


# x = np.concatenate(all_trials_x, axis=2)
# y = np.concatenate(all_trials_y, axis=2)

# plots_by_run = []
# for v in range(Ny):
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(np.arange(T)), 
#                       y=np.squeeze(x[:,:,v]), 
#                       mode='lines')
#     )
    
# plt(plots_by_run)

In [ ]:
# # Define a plotting function for callback that shows the current transition function estimate
# def callback_plot(pvec):
#     (mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J) = \
#         vec_to_params(pvec, y.shape[0], D, Nz, Ns)

#     # Plot transition function
#     xstar = np.arange(-2.0,4.0,0.05)

#     fp_get_static_K, fp_predict = create_fp_gp_funcs()
#     L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, sig_eps=Sigma_eps)
#     mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

#     plt([plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star), mode='markers'),
#          plt_type.Scatter(x=np.squeeze(z), y=np.squeeze(-2.0*np.ones_like(z)), mode='markers') \
#         ])

In [ ]:
# # Try to solve the full problem
# D = 1
# Nz = 8
# Ns = 1

# (mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J) = \
#     init_params(y, D, Nz, Ns)

In [ ]:
# (init_paramvec, dict_ind, dict_shape) = params_to_vec(mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J)
    
# # # Optimise only certain elements of paramvec
# #opt_params = np.concatenate([dict_ind['u'], dict_ind['s']])
# #cur_pvec = paramvec[opt_params] + np.random.randn(*(paramvec[opt_params].shape))

# tmp_func = lambda pvec: (time_full_iter(pvec, y, D, Nz, Ns)[0])
# objective_with_grad = value_and_grad(tmp_func, argnum=0)

# cur_pvec = init_paramvec

# # Add bounds to ensure positive variances
# bnds = list(((None, None),) * paramvec.shape[0])
# for i in np.concatenate([dict_ind['Sigma_0_0'], dict_ind['Sigma_nu'], dict_ind['Sigma_eps']]):
#     bnds[i] = (1e-6, None)
# bnds = tuple(bnds)
    
# result = scipy.optimize.minimize(objective_with_grad, cur_pvec, jac=True, method='L-BFGS-B', bounds=bnds, callback=callback_plot,
#                       options={'maxiter':50, 'disp':True})


In [ ]:
# #Plot likelihood wrt to fix point location only (other true parameters are given)
# D = 1
# Nz = 10
# Ns = 1

# # (mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J) = \
# #     init_params(y, D, Nz, Ns)
  

# mu_0_0 = np.array([[mu_0_0]])
# Sigma_0_0 = np.array([[Sigma_0_0]])
# C = np.array([[1]])
# Sigma_nu = np.array([[Sigma_nu]])
# z = np.concatenate(
#      [np.linspace(np.min(y),0.3,10)[:,None].T,
#       np.linspace(2.2,np.max(y),10)[:,None].T],
#     axis=1)
# u = z
# #Sigma_eps = np.array([[1e-10]])
# Sigma_eps = np.array([[Sigma_eps]])
# lengthscales = 0.6*np.ones((1,1))
# kernel_variance = 1.0*np.ones((1,1))
# J = np.zeros((1,1,1))
# s = np.array([[well_loc]])

# Nz = z.shape[1]
      
# # est_locs = np.linspace(0.8, 1.7, 10)
# # #est_locs = [1.32]
# # results = []
# # for est_well_loc in est_locs:
# #     s = np.array([[est_well_loc]])
# #     paramvec = params_to_vec(mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J)
    
# #     a = np.asscalar(time_full_iter(paramvec, y, D, Nz, Ns)[0])
# #     results.append(a)
    


In [ ]:
# well_loc_plot = [plt_type.Scatter(
#                       x=np.squeeze(est_locs), 
#                       y=np.squeeze(np.array(results)), 
#                       mode='markers')]

# plt(well_loc_plot)

In [ ]:
# s = np.array([[well_loc]])

# mu_0_0 = np.array([[-0.2]]) # Give wrong initial param
# paramvec = params_to_vec(mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J)[0]

# # Try to run proper optimisation on mu_0_0
# cur_pvec = paramvec[0:1]
# objective_with_grad = value_and_grad(lambda pvec: 
#                                          time_full_iter(np.concatenate([pvec, paramvec[1:]]), 
#                                                         y, D, Nz, Ns)[0], argnum=0)


# # # Test at various locations
# # est_mus = np.linspace(-0.3, 1.3, 40)
# # results = []
# # for est_mu in est_mus:
# #     mu_0_0 = np.array([[est_mu]])
# #     paramvec = params_to_vec(mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J)[0]
    
# #     a = np.asscalar(time_full_iter(paramvec, y, D, Nz, Ns)[0])
# #     results.append(a)


# # Optimise via manual gradient descent
# for it in range(10):
#     a = objective_with_grad(cur_pvec)
    
#     print [a, cur_pvec]
#     cur_pvec = (cur_pvec 
#                 - 1e-3*a[1]
#                 )


# # # Optimise via fancy gradient descent
# # result = scipy.optimize.minimize(objective_with_grad, cur_pvec, jac=True, method='CG',
# #                       options={'maxiter': 2, 'disp':True})

In [ ]:
# well_loc_plot = [plt_type.Scatter(
#                       x=np.squeeze(est_mus), 
#                       y=np.squeeze(np.array(results)), 
#                       mode='markers')]

# plt(well_loc_plot)

In [ ]:
# # Plot smoothed trajectories
# obj, x_t1, x_t, sig_t1, sig_t = time_full_iter(paramvec, y, D, Nz, Ns, ret_smoothed=True)


# plots_by_run = []
# for v in [1]:
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(np.arange(T)), 
#                       y=np.squeeze(y[:,:,v]), 
#                       mode='lines')
#     )
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(np.arange(T)), 
#                       y=np.squeeze(x[:,:,v]), 
#                       mode='lines')
#     )
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(np.arange(T)), 
#                       y=np.squeeze(x_t[:,:,v]), 
#                       mode='lines')
#     )
    
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(np.arange(T)), 
#                       y=np.squeeze(x_t1[:,:,v]), 
#                       mode='lines')
#     )
    
# plt(plots_by_run)


In [ ]:
# (mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_eps, lengthscales, kernel_variance, s, J) = \
#     vec_to_params(result.x, y.shape[0], D, Nz, Ns)

# # Plot transition function
# xstar = np.arange(-1.0,3.0,0.01)

# fp_get_static_K, fp_predict = create_fp_gp_funcs()
# L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, sig_eps=Sigma_eps)
# mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

# plt([plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star), mode='markers') #,
#      #plt_type.Scatter(x=np.squeeze(s), y=np.squeeze(s), mode='markers') \
#     ])